In [ ]:
#import data processing libraries

import os
import pandas as pd
import numpy as np
import math as math
import datetime as dt
from scipy import stats

#db connection libraries
import psycopg2
from sqlalchemy import create_engine
import psycopg2.extras as extras

In [ ]:
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)

#suppress scientific notation to 2 decimal places

pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
host = '157.245.248.249'
port = int(5432)
dbname = 'ubuntu'
user = 'jacklinengenia'
password = 'x3MX&8#!'


def get_query_results_postgres():
    with psycopg2.connect(host = host,
                          port = port,
                          database = dbname,
                          user = user,
                          password = password) as conn:
        sql = "SELECT * FROM bloomlive.idm_refresh_view"
        df = pd.read_sql(sql, conn)
        
    return df

    conn.close()
    
if __name__ == "__main__":
    get_query_results_postgres() 

In [ ]:
idm_df = get_query_results_postgres()

idm_df.head(3)

,id,idnumber,mydpdl90d,maxloanamamountlast6months,nownloans,nrepaidloans,nrepaidloanshighutiliz,maxowndpdever,recommendeddecision,creditlimit7days,creditlimit30days,cipscore,cipriskgrade,mobilescore,mobilescoreriskgrade,idm_refresh_date,created_at,full_name,date_of_birth,age,gender,marital_status,employment_status,reference_number,rnk
0,85996,00000000,NaN,NaN,NaN,NaN,NaN,NaN,Reject,NaN,NaN,NaN,None,NaN,None,2022-06-18,2022-06-19 12:39:55.131599,Subject Not Found in IPRS and CBS,None,None,None,None,None,31769321-0,1
1,176400,000193,NaN,NaN,NaN,NaN,NaN,NaN,Reject,NaN,NaN,NaN,None,NaN,None,2022-07-05,2022-07-06 12:22:00.039305,Subject Not Found in IPRS and CBS,None,None,None,None,None,32113515-941433372,1
2,192877,0006255,NaN,NaN,NaN,NaN,NaN,NaN,Approve,0.00,0.00,NaN,None,NaN,None,2022-07-19,2022-07-20 12:22:19.238376,Felix Kibue,1951-12-30,70,Male,NotSpecified,NotSpecified,32314943-792461395,1


In [ ]:
#rename columns

idm_df.rename(columns = {"idnumber": "national_id",
                        "recommendeddecision": "IDM_recommendation",
                        "creditlimit30days": "IDM_limit"}, inplace = True)

In [ ]:
#trim df to only the relevant columns

idm_df = idm_df[["national_id","IDM_recommendation","IDM_limit"]]

In [ ]:
# remove any rows without a national id
idm_df = idm_df[idm_df["national_id"].notnull()]


#convert national id to string column type
idm_df["national_id"] = idm_df["national_id"].astype(str)

In [ ]:
#remove duplicate national_id column

idm_df = idm_df.loc[:,~idm_df.columns.duplicated()]

In [ ]:
#filter out rows where "IDM_recommendation" column is blank

idm_df = idm_df.loc[idm_df["IDM_recommendation"].notnull()]

In [ ]:
#remove any whitespaces that may cause issues for specific column

idm_df["national_id"] = idm_df["national_id"].apply(lambda x: x.split(".")[0])
idm_df["national_id"] = idm_df["national_id"].apply(lambda x: x.split(" ")[0])

In [ ]:
idm_df.shape

(122694, 3)

In [ ]:
idm_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122694 entries, 0 to 122714
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   national_id         122694 non-null  object 
 1   IDM_recommendation  122694 non-null  object 
 2   IDM_limit           99340 non-null   float64
dtypes: float64(1), object(2)
memory usage: 3.7+ MB


In [ ]:
idm_df['IDM_recommendation'].value_counts()

Reject     87298
Approve    35396
Name: IDM_recommendation, dtype: int64

In [ ]:
host = '157.245.248.249'
port = int(5432)
dbname = 'ubuntu'
user = 'jacklinengenia'
password = 'x3MX&8#!'


def get_query_results_postgres():
    with psycopg2.connect(host = host,
                          port = port,
                          database = dbname,
                          user = user,
                          password = password) as conn:
        sql = "select * from bloomlive.client_summary_view csv2 where is_iprs_validated is true"
        df = pd.read_sql(sql, conn)
        
    return df

    conn.close()

In [ ]:
# loading data from scoring results table
store_nums_df = get_query_results_postgres()

store_nums_df.head(3)

,bloom_version,surrogate_id,mifos_id,mobile_number,store_number,national_id,first_name,middle_name,last_name,iprs_first_name,iprs_other_name,iprs_surname,date_of_birth,gender,status,submitted_on_date,client_type,company_name,provided_first_name,iprs_name_matched,is_iprs_checked,is_iprs_validated,primary_contact,secondary_contact,alternate_mobile_no1,alternate_mobile_no2,recommendeddecision,record_created_on_timestamp,pngme_uuid
0,2.00,108927,20627,254715870983,117126,2961228,Beatrice,Muthoni,Maranga,Beatrice,Muthoni,Maranga,1970-01-01,None,Active,2021-11-04,None,MEMOS BAR,BEATRICE,True,True,True,254715870983,254722577703,None,None,Approve,2022-01-03 16:05:36.978400,None
1,2.00,88315,15,254727091206,119180,25684857,Griffin,Gateri,Kariuki,Griffin,Gateri,Kariuki,1970-01-01,None,Active,2021-11-04,None,SPENCEREX GUARDS,GRIFFIN,True,True,True,254727091206,None,None,None,Reject,2022-01-03 16:05:36.978400,None
2,2.00,182023,99953,254727024590,124748,21746538,Augustine,Musyoka,Mutua,Lilian,Musangi,Mulwa,2022-05-24,None,Active,2022-05-24,None,AUGUSTINE MUSYOKA MUTUA MACHAKOS,None,False,True,True,254727024590,254720493964,None,None,Reject,2022-05-25 09:00:17.350618,None


In [ ]:
store_nums_df['bloom_version'].nunique()

2

In [ ]:
store_nums_df['is_iprs_validated'].nunique()

1

In [ ]:
store_nums_df.shape

(131140, 29)

In [ ]:
store_nums_df = store_nums_df.drop_duplicates(subset=['store_number'], keep='first')

store_nums_df.shape

(95235, 29)

In [ ]:
#load mapping for id and store numbers
#store_number_path = "C:\\Project_summaries\\Bloom\\Bloom all_loans\\20220721\\"


#load df
#store_nums_df = pd.read_excel(store_number_path+"client_summaries.xlsx")


#trim df to only target cols
store_nums_cols = ["national_id", "store_number", "is_iprs_validated"]
store_nums_df = store_nums_df[store_nums_cols]


#convert col to string
store_nums_df["national_id"] = store_nums_df["national_id"].astype("str")
store_nums_df["is_iprs_validated"] = store_nums_df["is_iprs_validated"].astype("str")

#store_nums_df["store_number"] = store_nums_df["store_number"].astype("str")



#clean up mobile number column
#store_nums_df[["national_id_2", "temp"]] = store_nums_df["national_id"].astype("str").str.split(".", expand=True)


#drop newly created temp column
#store_nums_df.drop(["temp","national_id"], axis=1, inplace=True)


#rename columns
#store_nums_df.rename(columns={"SHORT_CODE":"store_number","national_id":"national_id"}, inplace=True)

In [ ]:
store_nums_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95235 entries, 0 to 131137
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   national_id        95235 non-null  object
 1   store_number       95234 non-null  object
 2   is_iprs_validated  95235 non-null  object
dtypes: object(3)
memory usage: 2.9+ MB


In [ ]:
#left merge to assign idm df with store numbers

idm_df = pd.merge(idm_df, store_nums_df, how="left", on="national_id")

In [ ]:
idm_df.head(3)

,national_id,IDM_recommendation,IDM_limit,store_number,is_iprs_validated
0,00000000,Reject,NaN,NaN,NaN
1,000193,Reject,NaN,NaN,NaN
2,0006255,Approve,0.00,986801,True


In [ ]:
idm_df['IDM_recommendation'].value_counts()

Reject     90634
Approve    36839
Name: IDM_recommendation, dtype: int64

In [ ]:
idm_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 127473 entries, 0 to 127472
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   national_id         127473 non-null  object 
 1   IDM_recommendation  127473 non-null  object 
 2   IDM_limit           104079 non-null  float64
 3   store_number        95242 non-null   object 
 4   is_iprs_validated   95243 non-null   object 
dtypes: float64(1), object(4)
memory usage: 5.8+ MB


In [ ]:
idm_df.shape

(127473, 5)

---

#### Import Ubuntu scoring summary data

In [ ]:
#till_data path

#till_data_path = "C:\\Project_summaries\\Bloom\\Bloom all_loans\\20220721\\Till_data_summaries\\"

#load till df
ubuntu_scoring_summary_df = pd.read_excel("/content/Bloom_scoring_trx_data_20220914.xlsx")


#drop any extra unnecessary columns
ubuntu_scoring_summary_df = ubuntu_scoring_summary_df[ubuntu_scoring_summary_df.columns.drop(list(ubuntu_scoring_summary_df.filter(regex="Unnamed")))]


# remove any rows without a national id
ubuntu_scoring_summary_df = ubuntu_scoring_summary_df[ubuntu_scoring_summary_df["store_number"].notnull()]

In [ ]:
#create  copy of df

till_data = ubuntu_scoring_summary_df.copy()

#remove duplicate rows from till_data
till_duplicates = till_data.loc[till_data.duplicated()]
till_data = till_data.loc[~till_data.duplicated()]


#convert national_id column to string
till_data["national_id"] = till_data["national_id"].astype(str)


#remove any whitespaces that may cause issues for specific column
till_data["national_id"] = till_data["national_id"].apply(lambda x: x.split(".")[0])
till_data["national_id"] = till_data["national_id"].apply(lambda x: x.split(" ")[0])

#drop national id column
till_data.drop(columns="national_id", inplace=True)

In [ ]:
idm_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 127473 entries, 0 to 127472
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   national_id         127473 non-null  object 
 1   IDM_recommendation  127473 non-null  object 
 2   IDM_limit           104079 non-null  float64
 3   store_number        95242 non-null   object 
 4   is_iprs_validated   95243 non-null   object 
dtypes: float64(1), object(4)
memory usage: 5.8+ MB


In [ ]:
till_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99614 entries, 0 to 99613
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   mobile_number                      72273 non-null  object        
 1   store_number                       99614 non-null  int64         
 2   approx_30_days_trx_val             99614 non-null  float64       
 3   most_recent_trx_date_past_30_days  99614 non-null  datetime64[ns]
 4   last_trx_date                      99614 non-null  datetime64[ns]
 5   expected_trx_days                  99614 non-null  int64         
 6   actual_trx_days                    99614 non-null  int64         
 7   page_active_days                   99614 non-null  float64       
 8   inference_col                      99614 non-null  object        
 9   days_since_last_trx                99614 non-null  int64         
 10  transacted_last_5_days            

In [ ]:
till_data['store_number'] = till_data['store_number'].astype(str)

In [ ]:
#merge idm_df & till_data

df = till_data.merge(idm_df, how="left", on="store_number")

In [ ]:
df.head(3)

,mobile_number,store_number,approx_30_days_trx_val,most_recent_trx_date_past_30_days,last_trx_date,expected_trx_days,actual_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,IDM_recommendation,IDM_limit,is_iprs_validated
0,NaN,100207,1216.67,2022-06-17,2022-06-17,1,1,1.00,No_rules_relaxed,89,No,0.00,NaN,NaN,NaN,NaN
1,254711519875,101212,15000.00,2022-06-16,2022-09-11,88,25,0.28,No_rules_relaxed,3,Yes,1.00,8026792,Approve,5000.00,True
2,254716180492,105295,275045.62,2022-06-13,2022-09-12,92,86,0.93,relax_rules,2,Yes,1.00,27881033,Approve,4500.00,True


In [ ]:
df['IDM_recommendation'].value_counts()

Reject     47804
Approve    24473
Name: IDM_recommendation, dtype: int64

In [ ]:
df.shape

(99624, 16)

---

#### Load loans summary

In [ ]:
#load loans summary df

#path_2 = "C:\\Project_summaries\\Bloom\\Bloom all_loans\\20220721\\Analysis_summaries\\"

loans_summary = pd.read_excel("/content/Bloom_clients_loans_summary_20220914.xlsx")


#drop unnecassary column
loans_summary = loans_summary[loans_summary.columns.drop(list(loans_summary.filter(regex="Unnamed")))]

In [ ]:
loans_summary.head(3)

,client_mobile_number,store_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,inference_col,weight_dpd
0,254726604388.00,6,8,300,7,200000,2407.79,2022-07-30,2022-08-06,NaT,2022-08-06,38.00,2,active_default,491.00,200000,2022-07-04,5,0.62,45,No_rules_relaxed,0.00
1,NaN,11,3,600,30,5000,5000.00,2019-12-01,2019-12-31,2019-12-30,2019-12-31,-1.00,1,closed_early_repayment,NaN,5000,2019-10-27,3,1.00,1017,relax_rules,1.00
2,254714319251.00,30,14,300,7,2400,0.00,2022-09-12,2022-09-19,NaT,2022-09-19,-6.00,2,current_active,432.00,2400,2022-07-19,14,1.00,1,relax_rules,1.00


In [ ]:
loans_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47812 entries, 0 to 47811
Data columns (total 22 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   client_mobile_number              20493 non-null  float64       
 1   store_number                      47812 non-null  int64         
 2   loan_count                        47812 non-null  int64         
 3   loan_status                       47812 non-null  int64         
 4   term_frequency                    47812 non-null  int64         
 5   principal_disbursed               47812 non-null  int64         
 6   principal_repaid                  47812 non-null  float64       
 7   disbursed_on_date                 47812 non-null  datetime64[ns]
 8   expected_matured_on_date          47812 non-null  datetime64[ns]
 9   closed_on_date                    17836 non-null  datetime64[ns]
 10  due_date_fixed                    47812 non-nu

In [ ]:
#which customers do we have loans history but no till data summaries?

bloom_customers_no_till = list(set(loans_summary["store_number"]) - set(df["store_number"]))

In [ ]:
#which customers do we have loans history but no reported store_number?
bloom_customers_no_store_num = loans_summary[loans_summary["store_number"].isnull()][["client_mobile_number","store_number","bloom_version","loan_count","loan_repayment_status"]]


#remove customers whom we don't have tills for
#loans_summary = loans_summary[loans_summary["store_number"].notnull()]

In [ ]:
loans_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47812 entries, 0 to 47811
Data columns (total 22 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   client_mobile_number              20493 non-null  float64       
 1   store_number                      47812 non-null  int64         
 2   loan_count                        47812 non-null  int64         
 3   loan_status                       47812 non-null  int64         
 4   term_frequency                    47812 non-null  int64         
 5   principal_disbursed               47812 non-null  int64         
 6   principal_repaid                  47812 non-null  float64       
 7   disbursed_on_date                 47812 non-null  datetime64[ns]
 8   expected_matured_on_date          47812 non-null  datetime64[ns]
 9   closed_on_date                    17836 non-null  datetime64[ns]
 10  due_date_fixed                    47812 non-nu

In [ ]:
loans_summary['store_number'] = loans_summary['store_number'].astype(str)

In [ ]:
#merge filtered_df & till_data

df = pd.merge(df, loans_summary, how="left", on="store_number")

In [ ]:
df.shape

(99624, 37)

In [ ]:
#delete duplicate errors created because of broken ranking feature
df = df.sort_values("disbursed_on_date", ascending=True).drop_duplicates("store_number",keep='last')


#remove duplicate rows from df
df_duplicates = df.loc[df.duplicated()]
df = df.loc[~df.duplicated()]

In [ ]:
~df.duplicated()

92174    True
84778    True
1380     True
54930    True
98746    True
         ... 
99615    True
99616    True
99618    True
99619    True
99621    True
Length: 99614, dtype: bool

In [ ]:
df.shape

(99614, 37)

In [ ]:
#fillna loan_count column

cols_fillna = ["loan_count"]
# replace 'NaN' with zero in these columns
for col in cols_fillna:
    df[col].fillna(0,inplace=True)

In [ ]:
#fillna IDM_recommendation column

cols_fillna = ["IDM_recommendation"]
# replace 'NaN' with zero in these columns
for col in cols_fillna:
    df[col].fillna("Reject",inplace=True)

In [ ]:
df.drop(columns="inference_col_y", inplace=True)
df.rename(columns = {"inference_col_x":"inference_col"}, inplace=True)

In [ ]:
df['IDM_recommendation'].value_counts()

Reject     75143
Approve    24471
Name: IDM_recommendation, dtype: int64

In [ ]:
df.head()

,mobile_number,store_number,approx_30_days_trx_val,most_recent_trx_date_past_30_days,last_trx_date,expected_trx_days,actual_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,IDM_recommendation,IDM_limit,is_iprs_validated,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd
92174,254722978849,822369,156.67,2022-06-18,2022-06-18,1,1,1.00,relax_rules,88,No,0.00,22128407,Approve,0.00,True,NaN,4.00,600.00,7.00,5000.00,5000.00,2017-12-10,2017-12-17,2017-12-10,2017-12-17,-7.00,1.00,closed_early_repayment,NaN,5000.00,2017-12-10,4.00,1.00,1738.00,1.00
84778,254723173634,786152,2050.00,2022-06-14,2022-09-08,87,12,0.14,No_rules_relaxed,6,No,0.00,494024,Approve,0.00,True,NaN,1.00,601.00,30.00,12698.00,9206.00,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.00,1.00,written-off_default,NaN,12698.00,2017-12-18,0.00,0.00,1730.00,0.00
1380,254723916436,165978,810233.88,2022-06-13,2022-09-12,92,91,0.99,No_rules_relaxed,2,Yes,1.00,20417564,Approve,0.00,True,NaN,1.00,601.00,30.00,20000.00,12800.00,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.00,1.00,written-off_default,NaN,20000.00,2017-12-18,0.00,0.00,1730.00,0.00
54930,254720475133,745142,160453.67,2022-06-13,2022-09-12,92,89,0.97,No_rules_relaxed,2,Yes,1.00,22893884,Reject,0.00,True,NaN,1.00,601.00,30.00,28684.00,0.00,2017-12-21,2018-01-20,2019-12-31,2018-01-20,710.00,1.00,written-off_default,NaN,28684.00,2017-12-21,0.00,0.00,1727.00,0.00
98746,254724756423,978161,366.67,2022-07-12,2022-07-15,4,2,0.50,No_rules_relaxed,61,No,0.00,13261352,Approve,0.00,True,NaN,2.00,601.00,30.00,28600.00,0.00,2018-01-07,2018-02-06,2018-12-31,2018-02-06,328.00,1.00,written-off_default,NaN,28680.00,2017-12-19,1.00,0.50,1710.00,0.00


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99614 entries, 92174 to 99621
Data columns (total 36 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   mobile_number                      72273 non-null  object        
 1   store_number                       99614 non-null  object        
 2   approx_30_days_trx_val             99614 non-null  float64       
 3   most_recent_trx_date_past_30_days  99614 non-null  datetime64[ns]
 4   last_trx_date                      99614 non-null  datetime64[ns]
 5   expected_trx_days                  99614 non-null  int64         
 6   actual_trx_days                    99614 non-null  int64         
 7   page_active_days                   99614 non-null  float64       
 8   inference_col                      99614 non-null  object        
 9   days_since_last_trx                99614 non-null  int64         
 10  transacted_last_5_days        

#### Stabilisation & scoring adjustments

In [ ]:
#create new loan count column

df["adjusted_loan_count"] = df["loan_count"]

In [ ]:
#adjust loan count col

df.loc[(df["IDM_recommendation"]=="Reject")&(df["loan_count"]<6), "adjusted_loan_count"] = 0
df.loc[df["num_days_since_last_disbursement"]> 180, "adjusted_loan_count"] = 0

In [ ]:
def calc_limit_factor_21(df):
    
    idm_recommendation = df["IDM_recommendation"]
    CRB_approve_limit_factor = 0.50
    CRB_reject_limit_factor = 0.35
    
    if idm_recommendation == 'Approve':
        return CRB_approve_limit_factor
    else:
        return CRB_reject_limit_factor    
    
    
def calc_limit_factor_7(df):
    
    idm_recommendation = df["IDM_recommendation"]
    CRB_approve_limit_factor = 0.17
    CRB_reject_limit_factor = 0.12
    
    if idm_recommendation == 'Approve':
        return CRB_approve_limit_factor
    else:
        return CRB_reject_limit_factor
    
    
    
def calc_limit_factor_1(df):
    
    idm_recommendation = df["IDM_recommendation"]
    CRB_approve_limit_factor = 0.17
    CRB_reject_limit_factor = 0.12
    
    if idm_recommendation == 'Approve':
        return CRB_approve_limit_factor
    else:
        return CRB_reject_limit_factor

In [ ]:
df['limit_factor_21'] = df.apply(lambda x: calc_limit_factor_21(x), axis = 1)
df['limit_factor_7'] = df.apply(lambda x: calc_limit_factor_7(x), axis = 1)
df['limit_factor_1'] = df.apply(lambda x: calc_limit_factor_1(x), axis = 1)

df.head(3)

,mobile_number,store_number,approx_30_days_trx_val,most_recent_trx_date_past_30_days,last_trx_date,expected_trx_days,actual_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,IDM_recommendation,IDM_limit,is_iprs_validated,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1
92174,254722978849,822369,156.67,2022-06-18,2022-06-18,1,1,1.00,relax_rules,88,No,0.00,22128407,Approve,0.00,True,NaN,4.00,600.00,7.00,5000.00,5000.00,2017-12-10,2017-12-17,2017-12-10,2017-12-17,-7.00,1.00,closed_early_repayment,NaN,5000.00,2017-12-10,4.00,1.00,1738.00,1.00,0.00,0.50,0.17,0.17
84778,254723173634,786152,2050.00,2022-06-14,2022-09-08,87,12,0.14,No_rules_relaxed,6,No,0.00,494024,Approve,0.00,True,NaN,1.00,601.00,30.00,12698.00,9206.00,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.00,1.00,written-off_default,NaN,12698.00,2017-12-18,0.00,0.00,1730.00,0.00,0.00,0.50,0.17,0.17
1380,254723916436,165978,810233.88,2022-06-13,2022-09-12,92,91,0.99,No_rules_relaxed,2,Yes,1.00,20417564,Approve,0.00,True,NaN,1.00,601.00,30.00,20000.00,12800.00,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.00,1.00,written-off_default,NaN,20000.00,2017-12-18,0.00,0.00,1730.00,0.00,0.00,0.50,0.17,0.17


In [ ]:
# adding IDM_factor_21, IDM_factor_7, IDM_factor_1 columns to the dataframe
df['IDM_factor_21'] = df['limit_factor_21'] / 0.5
df['IDM_factor_7'] = df['limit_factor_7'] / 0.17
df['IDM_factor_1'] = df['limit_factor_1'] / 0.17

df.head(3)

,mobile_number,store_number,approx_30_days_trx_val,most_recent_trx_date_past_30_days,last_trx_date,expected_trx_days,actual_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,IDM_recommendation,IDM_limit,is_iprs_validated,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,IDM_factor_21,IDM_factor_7,IDM_factor_1
92174,254722978849,822369,156.67,2022-06-18,2022-06-18,1,1,1.00,relax_rules,88,No,0.00,22128407,Approve,0.00,True,NaN,4.00,600.00,7.00,5000.00,5000.00,2017-12-10,2017-12-17,2017-12-10,2017-12-17,-7.00,1.00,closed_early_repayment,NaN,5000.00,2017-12-10,4.00,1.00,1738.00,1.00,0.00,0.50,0.17,0.17,1.00,1.00,1.00
84778,254723173634,786152,2050.00,2022-06-14,2022-09-08,87,12,0.14,No_rules_relaxed,6,No,0.00,494024,Approve,0.00,True,NaN,1.00,601.00,30.00,12698.00,9206.00,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.00,1.00,written-off_default,NaN,12698.00,2017-12-18,0.00,0.00,1730.00,0.00,0.00,0.50,0.17,0.17,1.00,1.00,1.00
1380,254723916436,165978,810233.88,2022-06-13,2022-09-12,92,91,0.99,No_rules_relaxed,2,Yes,1.00,20417564,Approve,0.00,True,NaN,1.00,601.00,30.00,20000.00,12800.00,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.00,1.00,written-off_default,NaN,20000.00,2017-12-18,0.00,0.00,1730.00,0.00,0.00,0.50,0.17,0.17,1.00,1.00,1.00


In [ ]:
# defining a function for the limit factors

# defining a function to create bands for trading consistency
def calc_trading_consistency_bands(df):
    count = df['page_active_days']
    
    if count >= 0 and count < 0.30:
        return 'Band 1'
    elif count >= 0.30 and count < 0.50:
        return 'Band 2'
    elif count >= 0.50 and count < 0.60:
        return 'Band 3'
    elif count >= 0.60 and count < 0.70:
        return 'Band 4'
    elif count >= 0.70 and count < 0.80:
        return 'Band 5'
    elif count >= 0.80 and count < 0.90:
        return 'Band 6'
    elif count >= 0.90 and count <= 1.00:
        return 'Band 7'


# defining a function to create bands for loan count
def calc_loan_count_bands(df):
    count = df['loan_count']
  
    if count == 0:
        return 'Band 1'
    elif count == 1 or count == 2:
        return 'Band 2'
    elif count == 3 or count == 4:
        return 'Band 3'
    elif count == 5 or count == 6:
        return 'Band 4'
    elif count == 7 or count == 8:
        return 'Band 5'
    elif count == 9 or count == 10:
        return 'Band 6'
    elif count == 11 or count == 12:
        return 'Band 7'
    elif count > 12:
        return 'Band 8'


# defining a function to use the bands for both trading consistency and loan count to get limit factors
def calc_limit_factor_21(df):
    trading_consistency = df['trading_consistency_bands']
    loan_count = df['loan_count_bands']
  
    if (trading_consistency == 'Band 1' and loan_count == 'Band 1') or (trading_consistency == 'Band 1' and loan_count == 'Band 2') or (trading_consistency == 'Band 1' and loan_count == 'Band 3')  or (trading_consistency == 'Band 2' and loan_count == 'Band 1') or (trading_consistency == 'Band 2' and loan_count == 'Band 2') or (trading_consistency == 'Band 3' and loan_count == 'Band 1') or (trading_consistency == 'Band 4' and loan_count == 'Band 1') or (trading_consistency == 'Band 5' and loan_count == 'Band 1') or (trading_consistency == 'Band 6' and loan_count == 'Band 1') or (trading_consistency == 'Band 7' and loan_count == 'Band 1'):
        return 0.00
    elif (trading_consistency == 'Band 1' and loan_count == 'Band 4') or (trading_consistency == 'Band 1' and loan_count == 'Band 5') or (trading_consistency == 'Band 2' and loan_count == 'Band 3') or (trading_consistency == 'Band 2' and loan_count == 'Band 4') or (trading_consistency == 'Band 3' and loan_count == 'Band 2') or (trading_consistency == 'Band 3' and loan_count == 'Band 3'):
        return 0.05
    elif (trading_consistency == 'Band 1' and loan_count == 'Band 6') or (trading_consistency == 'Band 2' and loan_count == 'Band 5') or (trading_consistency == 'Band 3' and loan_count == 'Band 4') or (trading_consistency == 'Band 4' and loan_count == 'Band 2') or (trading_consistency == 'Band 4' and loan_count == 'Band 3'):
        return 0.10
    elif (trading_consistency == 'Band 1' and loan_count == 'Band 7') or (trading_consistency == 'Band 2' and loan_count == 'Band 6') or (trading_consistency == 'Band 3' and loan_count == 'Band 5') or (trading_consistency == 'Band 4' and loan_count == 'Band 4') or (trading_consistency == 'Band 5' and loan_count == 'Band 2') or (trading_consistency == 'Band 5' and loan_count == 'Band 3'):
        return 0.15
    elif (trading_consistency == 'Band 1' and loan_count == 'Band 8') or (trading_consistency == 'Band 2' and loan_count == 'Band 7') or (trading_consistency == 'Band 3' and loan_count == 'Band 6') or (trading_consistency == 'Band 4' and loan_count == 'Band 5') or (trading_consistency == 'Band 5' and loan_count == 'Band 4') or (trading_consistency == 'Band 6' and loan_count == 'Band 2'):
        return 0.20
    elif (trading_consistency == 'Band 2' and loan_count == 'Band 8') or (trading_consistency == 'Band 3' and loan_count == 'Band 7') or (trading_consistency == 'Band 4' and loan_count == 'Band 6') or (trading_consistency == 'Band 5' and loan_count == 'Band 5') or (trading_consistency == 'Band 6' and loan_count == 'Band 3') or (trading_consistency == 'Band 7' and loan_count == 'Band 2'):
        return 0.25
    elif (trading_consistency == 'Band 3' and loan_count == 'Band 8') or (trading_consistency == 'Band 4' and loan_count == 'Band 7') or (trading_consistency == 'Band 5' and loan_count == 'Band 6') or (trading_consistency == 'Band 6' and loan_count == 'Band 4') or (trading_consistency == 'Band 7' and loan_count == 'Band 3'):
        return 0.30
    elif (trading_consistency == 'Band 4' and loan_count == 'Band 8') or (trading_consistency == 'Band 5' and loan_count == 'Band 7') or (trading_consistency == 'Band 6' and loan_count == 'Band 5') or (trading_consistency == 'Band 7' and loan_count == 'Band 4'):
        return 0.35
    elif (trading_consistency == 'Band 5' and loan_count == 'Band 8') or (trading_consistency == 'Band 6' and loan_count == 'Band 6') or (trading_consistency == 'Band 7' and loan_count == 'Band 5'):
        return 0.40
    elif (trading_consistency == 'Band 6' and loan_count == 'Band 7') or (trading_consistency == 'Band 7' and loan_count == 'Band 6'):
        return 0.45
    elif (trading_consistency == 'Band 6' and loan_count == 'Band 8') or (trading_consistency == 'Band 7' and loan_count == 'Band 7'):
        return 0.50
    elif trading_consistency == 'Band 7' and loan_count == 'Band 8':
        return 0.55



def calc_limit_factor_7(df):
    trading_consistency = df['trading_consistency_bands']
    loan_count = df['loan_count_bands']
  
    if (trading_consistency == 'Band 1' and loan_count == 'Band 1') or (trading_consistency == 'Band 1' and loan_count == 'Band 2') or (trading_consistency == 'Band 1' and loan_count == 'Band 3') or (trading_consistency == 'Band 2' and loan_count == 'Band 1') or (trading_consistency == 'Band 2' and loan_count == 'Band 2'):
        return 0.00
    elif (trading_consistency == 'Band 1' and loan_count == 'Band 4') or (trading_consistency == 'Band 1' and loan_count == 'Band 5') or (trading_consistency == 'Band 2' and loan_count == 'Band 3') or (trading_consistency == 'Band 3' and loan_count == 'Band 1') or (trading_consistency == 'Band 3' and loan_count == 'Band 2') or (trading_consistency == 'Band 4' and loan_count == 'Band 1'):
        return 0.075
    elif (trading_consistency == 'Band 1' and loan_count == 'Band 6') or (trading_consistency == 'Band 2' and loan_count == 'Band 4') or (trading_consistency == 'Band 2' and loan_count == 'Band 5') or (trading_consistency == 'Band 3' and loan_count == 'Band 3') or (trading_consistency == 'Band 4' and loan_count == 'Band 2') or (trading_consistency == 'Band 5' and loan_count == 'Band 1'):
        return 0.10
    elif (trading_consistency == 'Band 1' and loan_count == 'Band 7') or (trading_consistency == 'Band 2' and loan_count == 'Band 6') or (trading_consistency == 'Band 3' and loan_count == 'Band 4') or (trading_consistency == 'Band 4' and loan_count == 'Band 3') or (trading_consistency == 'Band 5' and loan_count == 'Band 2') or (trading_consistency == 'Band 6' and loan_count == 'Band 1') or (trading_consistency == 'Band 7' and loan_count == 'Band 1'):
        return 0.125
    elif (trading_consistency == 'Band 1' and loan_count == 'Band 8') or (trading_consistency == 'Band 2' and loan_count == 'Band 7') or (trading_consistency == 'Band 3' and loan_count == 'Band 5') or (trading_consistency == 'Band 3' and loan_count == 'Band 6') or (trading_consistency == 'Band 4' and loan_count == 'Band 4') or (trading_consistency == 'Band 4' and loan_count == 'Band 5') or (trading_consistency == 'Band 5' and loan_count == 'Band 3') or (trading_consistency == 'Band 6' and loan_count == 'Band 2') or (trading_consistency == 'Band 6' and loan_count == 'Band 3') or (trading_consistency == 'Band 7' and loan_count == 'Band 2'):
        return 0.15
    elif (trading_consistency == 'Band 2' and loan_count == 'Band 8') or (trading_consistency == 'Band 3' and loan_count == 'Band 7') or (trading_consistency == 'Band 4' and loan_count == 'Band 6') or (trading_consistency == 'Band 5' and loan_count == 'Band 4') or (trading_consistency == 'Band 5' and loan_count == 'Band 5') or (trading_consistency == 'Band 6' and loan_count == 'Band 4') or (trading_consistency == 'Band 7' and loan_count == 'Band 3'):
        return 0.175
    elif (trading_consistency == 'Band 3' and loan_count == 'Band 8') or (trading_consistency == 'Band 4' and loan_count == 'Band 7') or (trading_consistency == 'Band 5' and loan_count == 'Band 6') or (trading_consistency == 'Band 6' and loan_count == 'Band 5') or (trading_consistency == 'Band 7' and loan_count == 'Band 4'):
        return 0.20
    elif (trading_consistency == 'Band 4' and loan_count == 'Band 8') or (trading_consistency == 'Band 5' and loan_count == 'Band 7') or (trading_consistency == 'Band 6' and loan_count == 'Band 6') or (trading_consistency == 'Band 7' and loan_count == 'Band 5'):
        return 0.225
    elif (trading_consistency == 'Band 5' and loan_count == 'Band 8') or (trading_consistency == 'Band 6' and loan_count == 'Band 7') or (trading_consistency == 'Band 7' and loan_count == 'Band 6'):
        return 0.25
    elif (trading_consistency == 'Band 6' and loan_count == 'Band 8') or (trading_consistency == 'Band 7' and loan_count == 'Band 7'):
        return 0.275
    elif (trading_consistency == 'Band 7' and loan_count == 'Band 8'):
        return 0.30




def calc_limit_factor_1(df):
    trading_consistency = df['trading_consistency_bands']
    loan_count = df['loan_count_bands']
  
    if (trading_consistency == 'Band 1' and loan_count == 'Band 1') or (trading_consistency == 'Band 1' and loan_count == 'Band 2') or (trading_consistency == 'Band 1' and loan_count == 'Band 3') or (trading_consistency == 'Band 2' and loan_count == 'Band 1') or (trading_consistency == 'Band 2' and loan_count == 'Band 2'):
        return 0.00
    elif (trading_consistency == 'Band 1' and loan_count == 'Band 4') or (trading_consistency == 'Band 1' and loan_count == 'Band 5') or (trading_consistency == 'Band 2' and loan_count == 'Band 3') or (trading_consistency == 'Band 3' and loan_count == 'Band 1') or (trading_consistency == 'Band 3' and loan_count == 'Band 2') or (trading_consistency == 'Band 4' and loan_count == 'Band 1'):
        return 0.075
    elif (trading_consistency == 'Band 1' and loan_count == 'Band 6') or (trading_consistency == 'Band 2' and loan_count == 'Band 4') or (trading_consistency == 'Band 2' and loan_count == 'Band 5') or (trading_consistency == 'Band 3' and loan_count == 'Band 3') or (trading_consistency == 'Band 4' and loan_count == 'Band 2') or (trading_consistency == 'Band 5' and loan_count == 'Band 1'):
        return 0.10
    elif (trading_consistency == 'Band 1' and loan_count == 'Band 7') or (trading_consistency == 'Band 2' and loan_count == 'Band 6') or (trading_consistency == 'Band 3' and loan_count == 'Band 4') or (trading_consistency == 'Band 4' and loan_count == 'Band 3') or (trading_consistency == 'Band 5' and loan_count == 'Band 2') or (trading_consistency == 'Band 6' and loan_count == 'Band 1') or (trading_consistency == 'Band 7' and loan_count == 'Band 1'):
        return 0.125
    elif (trading_consistency == 'Band 1' and loan_count == 'Band 8') or (trading_consistency == 'Band 2' and loan_count == 'Band 7') or (trading_consistency == 'Band 3' and loan_count == 'Band 5') or (trading_consistency == 'Band 3' and loan_count == 'Band 6') or (trading_consistency == 'Band 4' and loan_count == 'Band 4') or (trading_consistency == 'Band 4' and loan_count == 'Band 5') or (trading_consistency == 'Band 5' and loan_count == 'Band 3') or (trading_consistency == 'Band 6' and loan_count == 'Band 2') or (trading_consistency == 'Band 6' and loan_count == 'Band 3') or (trading_consistency == 'Band 7' and loan_count == 'Band 2'):
        return 0.15
    elif (trading_consistency == 'Band 2' and loan_count == 'Band 8') or (trading_consistency == 'Band 3' and loan_count == 'Band 7') or (trading_consistency == 'Band 4' and loan_count == 'Band 6') or (trading_consistency == 'Band 5' and loan_count == 'Band 4') or (trading_consistency == 'Band 5' and loan_count == 'Band 5') or (trading_consistency == 'Band 6' and loan_count == 'Band 4') or (trading_consistency == 'Band 7' and loan_count == 'Band 3'):
        return 0.175
    elif (trading_consistency == 'Band 3' and loan_count == 'Band 8') or (trading_consistency == 'Band 4' and loan_count == 'Band 7') or (trading_consistency == 'Band 5' and loan_count == 'Band 6') or (trading_consistency == 'Band 6' and loan_count == 'Band 5') or (trading_consistency == 'Band 7' and loan_count == 'Band 4'):
        return 0.20
    elif (trading_consistency == 'Band 4' and loan_count == 'Band 8') or (trading_consistency == 'Band 5' and loan_count == 'Band 7') or (trading_consistency == 'Band 6' and loan_count == 'Band 6') or (trading_consistency == 'Band 7' and loan_count == 'Band 5'):
        return 0.225
    elif (trading_consistency == 'Band 5' and loan_count == 'Band 8') or (trading_consistency == 'Band 6' and loan_count == 'Band 7') or (trading_consistency == 'Band 7' and loan_count == 'Band 6'):
        return 0.25
    elif (trading_consistency == 'Band 6' and loan_count == 'Band 8') or (trading_consistency == 'Band 7' and loan_count == 'Band 7'):
        return 0.275
    elif (trading_consistency == 'Band 7' and loan_count == 'Band 8'):
        return 0.30

In [ ]:
# adding trading_consistency_bands, loan_count_bands and limit_factor columns to the dataframe by applying the functions to the dataframe
df['trading_consistency_bands'] = df.apply(lambda x: calc_trading_consistency_bands(x), axis = 1)
df['loan_count_bands'] = df.apply(lambda x: calc_loan_count_bands(x), axis = 1)

df['new_limit_factor_21'] = df.apply(lambda x: calc_limit_factor_21(x), axis = 1)
df['new_limit_factor_7'] = df.apply(lambda x: calc_limit_factor_7(x), axis = 1)
df['new_limit_factor_1'] = df.apply(lambda x: calc_limit_factor_1(x), axis = 1)

df.head(3)

,mobile_number,store_number,approx_30_days_trx_val,most_recent_trx_date_past_30_days,last_trx_date,expected_trx_days,actual_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,IDM_recommendation,IDM_limit,is_iprs_validated,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,IDM_factor_21,IDM_factor_7,IDM_factor_1,trading_consistency_bands,loan_count_bands,new_limit_factor_21,new_limit_factor_7,new_limit_factor_1
92174,254722978849,822369,156.67,2022-06-18,2022-06-18,1,1,1.00,relax_rules,88,No,0.00,22128407,Approve,0.00,True,NaN,4.00,600.00,7.00,5000.00,5000.00,2017-12-10,2017-12-17,2017-12-10,2017-12-17,-7.00,1.00,closed_early_repayment,NaN,5000.00,2017-12-10,4.00,1.00,1738.00,1.00,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 7,Band 3,0.30,0.17,0.17
84778,254723173634,786152,2050.00,2022-06-14,2022-09-08,87,12,0.14,No_rules_relaxed,6,No,0.00,494024,Approve,0.00,True,NaN,1.00,601.00,30.00,12698.00,9206.00,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.00,1.00,written-off_default,NaN,12698.00,2017-12-18,0.00,0.00,1730.00,0.00,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 1,Band 2,0.00,0.00,0.00
1380,254723916436,165978,810233.88,2022-06-13,2022-09-12,92,91,0.99,No_rules_relaxed,2,Yes,1.00,20417564,Approve,0.00,True,NaN,1.00,601.00,30.00,20000.00,12800.00,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.00,1.00,written-off_default,NaN,20000.00,2017-12-18,0.00,0.00,1730.00,0.00,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 7,Band 2,0.25,0.15,0.15


In [ ]:
def cal_days_past_due(df):
    days_past_due = df['days_past_due']
    loan_count = df['loan_count']
    
    if loan_count == 0:
        return 1
    else:
        return days_past_due


def cal_weight_dpd(df):
    
    loan_count = df['loan_count']
    days_past_due = df['days_past_due']
    
    if days_past_due <= 0:
        return 1
    else:
        return 0

    
    
def cal_good_loans_repayment_ratio(df):
    good_loans_repayment_ratio = df['good_loans_repayment_ratio']
    loan_count = df['loan_count']
    
    if loan_count == 0:
        return 1
    else:
        return good_loans_repayment_ratio


def cal_weight_good_loans_repayment_ratio(df):
    good_loans_repayment_ratio = df['good_loans_repayment_ratio']
    
    if good_loans_repayment_ratio >= 1:
        return good_loans_repayment_ratio
    else:
        return 0      

    
# defining a function for consistency weights
def calc_weight_consistency(df):
    page_active_days = df['page_active_days']
    inference_col = df["inference_col"]
  
    if page_active_days >= 0.85:
        return 1.0
    else:
        return 0
            
    
    
def cal_weight_recency(df):
    days_since_last_trx = df['days_since_last_trx']
    
    if days_since_last_trx <= 3:
        return days_since_last_trx
    else:
        return 0  


In [ ]:
df['days_past_due'] = df.apply(lambda x: cal_days_past_due(x), axis = 1)
df['weight_dpd'] = df.apply(lambda x: cal_weight_dpd(x), axis = 1)
df['good_loans_repayment_ratio'] = df.apply(lambda x: cal_good_loans_repayment_ratio(x), axis = 1)
df['weight_good_loans_repayment_ratio'] = df.apply(lambda x: cal_weight_good_loans_repayment_ratio(x), axis = 1)
df['weight_consistency'] = df.apply(lambda x: calc_weight_consistency(x), axis = 1)
df['weight_recency'] = df.apply(lambda x: cal_weight_recency(x), axis = 1)


df.head(3)

,mobile_number,store_number,approx_30_days_trx_val,most_recent_trx_date_past_30_days,last_trx_date,expected_trx_days,actual_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,IDM_recommendation,IDM_limit,is_iprs_validated,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,IDM_factor_21,IDM_factor_7,IDM_factor_1,trading_consistency_bands,loan_count_bands,new_limit_factor_21,new_limit_factor_7,new_limit_factor_1,weight_good_loans_repayment_ratio,weight_consistency,weight_recency
92174,254722978849,822369,156.67,2022-06-18,2022-06-18,1,1,1.00,relax_rules,88,No,0.00,22128407,Approve,0.00,True,NaN,4.00,600.00,7.00,5000.00,5000.00,2017-12-10,2017-12-17,2017-12-10,2017-12-17,-7.00,1.00,closed_early_repayment,NaN,5000.00,2017-12-10,4.00,1.00,1738.00,1,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 7,Band 3,0.30,0.17,0.17,1.00,1.00,0
84778,254723173634,786152,2050.00,2022-06-14,2022-09-08,87,12,0.14,No_rules_relaxed,6,No,0.00,494024,Approve,0.00,True,NaN,1.00,601.00,30.00,12698.00,9206.00,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.00,1.00,written-off_default,NaN,12698.00,2017-12-18,0.00,0.00,1730.00,0,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 1,Band 2,0.00,0.00,0.00,0.00,0.00,0
1380,254723916436,165978,810233.88,2022-06-13,2022-09-12,92,91,0.99,No_rules_relaxed,2,Yes,1.00,20417564,Approve,0.00,True,NaN,1.00,601.00,30.00,20000.00,12800.00,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.00,1.00,written-off_default,NaN,20000.00,2017-12-18,0.00,0.00,1730.00,0,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 7,Band 2,0.25,0.15,0.15,0.00,1.00,2


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99614 entries, 92174 to 99621
Data columns (total 51 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   mobile_number                      72273 non-null  object        
 1   store_number                       99614 non-null  object        
 2   approx_30_days_trx_val             99614 non-null  float64       
 3   most_recent_trx_date_past_30_days  99614 non-null  datetime64[ns]
 4   last_trx_date                      99614 non-null  datetime64[ns]
 5   expected_trx_days                  99614 non-null  int64         
 6   actual_trx_days                    99614 non-null  int64         
 7   page_active_days                   99614 non-null  float64       
 8   inference_col                      99614 non-null  object        
 9   days_since_last_trx                99614 non-null  int64         
 10  transacted_last_5_days        

In [ ]:
# adding risk_rules_factor column to the dataframe
df['risk_rules_factor'] = (df['weight_dpd'] + df['weight_recency'] + df['weight_consistency'] + df['weight_good_loans_repayment_ratio']) / 4


df.head(3)

,mobile_number,store_number,approx_30_days_trx_val,most_recent_trx_date_past_30_days,last_trx_date,expected_trx_days,actual_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,IDM_recommendation,IDM_limit,is_iprs_validated,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,IDM_factor_21,IDM_factor_7,IDM_factor_1,trading_consistency_bands,loan_count_bands,new_limit_factor_21,new_limit_factor_7,new_limit_factor_1,weight_good_loans_repayment_ratio,weight_consistency,weight_recency,risk_rules_factor
92174,254722978849,822369,156.67,2022-06-18,2022-06-18,1,1,1.00,relax_rules,88,No,0.00,22128407,Approve,0.00,True,NaN,4.00,600.00,7.00,5000.00,5000.00,2017-12-10,2017-12-17,2017-12-10,2017-12-17,-7.00,1.00,closed_early_repayment,NaN,5000.00,2017-12-10,4.00,1.00,1738.00,1,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 7,Band 3,0.30,0.17,0.17,1.00,1.00,0,0.75
84778,254723173634,786152,2050.00,2022-06-14,2022-09-08,87,12,0.14,No_rules_relaxed,6,No,0.00,494024,Approve,0.00,True,NaN,1.00,601.00,30.00,12698.00,9206.00,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.00,1.00,written-off_default,NaN,12698.00,2017-12-18,0.00,0.00,1730.00,0,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 1,Band 2,0.00,0.00,0.00,0.00,0.00,0,0.00
1380,254723916436,165978,810233.88,2022-06-13,2022-09-12,92,91,0.99,No_rules_relaxed,2,Yes,1.00,20417564,Approve,0.00,True,NaN,1.00,601.00,30.00,20000.00,12800.00,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.00,1.00,written-off_default,NaN,20000.00,2017-12-18,0.00,0.00,1730.00,0,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 7,Band 2,0.25,0.15,0.15,0.00,1.00,2,0.75


In [ ]:
df['good_loans_repayment_ratio'].mean()

0.878523199550264

In [ ]:
# adding ultimate_factor_21, ultimate_factor_7, ultimate_factor_1 columns to the dataframe
df['ultimate_factor_21'] = df['risk_rules_factor'] * df['IDM_factor_21'] * df['new_limit_factor_21']
df['ultimate_factor_7'] = df['risk_rules_factor'] * df['IDM_factor_7'] * df['new_limit_factor_7']
df['ultimate_factor_1'] = df['risk_rules_factor'] * df['IDM_factor_1'] * df['new_limit_factor_1']

In [ ]:
df['limit_21_day'] = df['approx_30_days_trx_val'] * df['ultimate_factor_21']
df['limit_7_day'] = df['approx_30_days_trx_val'] * df['ultimate_factor_7']
df['limit_1_day'] = df['approx_30_days_trx_val'] * df['ultimate_factor_1']

df.head(3)

,mobile_number,store_number,approx_30_days_trx_val,most_recent_trx_date_past_30_days,last_trx_date,expected_trx_days,actual_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,IDM_recommendation,IDM_limit,is_iprs_validated,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,IDM_factor_21,IDM_factor_7,IDM_factor_1,trading_consistency_bands,loan_count_bands,new_limit_factor_21,new_limit_factor_7,new_limit_factor_1,weight_good_loans_repayment_ratio,weight_consistency,weight_recency,risk_rules_factor,ultimate_factor_21,ultimate_factor_7,ultimate_factor_1,limit_21_day,limit_7_day,limit_1_day
92174,254722978849,822369,156.67,2022-06-18,2022-06-18,1,1,1.00,relax_rules,88,No,0.00,22128407,Approve,0.00,True,NaN,4.00,600.00,7.00,5000.00,5000.00,2017-12-10,2017-12-17,2017-12-10,2017-12-17,-7.00,1.00,closed_early_repayment,NaN,5000.00,2017-12-10,4.00,1.00,1738.00,1,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 7,Band 3,0.30,0.17,0.17,1.00,1.00,0,0.75,0.22,0.13,0.13,35.25,20.56,20.56
84778,254723173634,786152,2050.00,2022-06-14,2022-09-08,87,12,0.14,No_rules_relaxed,6,No,0.00,494024,Approve,0.00,True,NaN,1.00,601.00,30.00,12698.00,9206.00,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.00,1.00,written-off_default,NaN,12698.00,2017-12-18,0.00,0.00,1730.00,0,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 1,Band 2,0.00,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1380,254723916436,165978,810233.88,2022-06-13,2022-09-12,92,91,0.99,No_rules_relaxed,2,Yes,1.00,20417564,Approve,0.00,True,NaN,1.00,601.00,30.00,20000.00,12800.00,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.00,1.00,written-off_default,NaN,20000.00,2017-12-18,0.00,0.00,1730.00,0,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 7,Band 2,0.25,0.15,0.15,0.00,1.00,2,0.75,0.19,0.11,0.11,151918.85,91151.31,91151.31


In [ ]:
df['IDM_recommendation'].value_counts()

Reject     75143
Approve    24471
Name: IDM_recommendation, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99614 entries, 92174 to 99621
Data columns (total 58 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   mobile_number                      72273 non-null  object        
 1   store_number                       99614 non-null  object        
 2   approx_30_days_trx_val             99614 non-null  float64       
 3   most_recent_trx_date_past_30_days  99614 non-null  datetime64[ns]
 4   last_trx_date                      99614 non-null  datetime64[ns]
 5   expected_trx_days                  99614 non-null  int64         
 6   actual_trx_days                    99614 non-null  int64         
 7   page_active_days                   99614 non-null  float64       
 8   inference_col                      99614 non-null  object        
 9   days_since_last_trx                99614 non-null  int64         
 10  transacted_last_5_days        

In [ ]:
df["inference_col"].value_counts()

No_rules_relaxed    84255
relax_rules         15359
Name: inference_col, dtype: int64

In [ ]:
def limits_decrease_zeroization_21(df):
    """
    function to adjust allocated limits in line with past repayment behavior
    
    Inputs:
    1) num of loans taken,
    2) previously allocated limit,
    3) term frequency for a loan,
    4) days past due for the most recent loan
    5) good loans repayment ratio
    6) inference variable i.e declaring whether borrower qualifies for limit stabilization OR not
    
    Output:
    adjusted limit based on rollover/default patterns of most recent loan
    """
    dpd_allowance = 10
    repayment_ratio_threshold = 0.6
    zeroize = 0
    
    #loan_count = df["loan_count"]
    loan_count = df["adjusted_loan_count"]
    limit_col = df["limit_21_day"]
    latest_loan_dpd = df["days_past_due"]
    term_frequency = df["term_frequency"]
    repayment_ratio = df["good_loans_repayment_ratio"]
    inference_col = df["inference_col"]
    
    
    conditions = [
        loan_count.isna(),
        loan_count.eq(0),
        inference_col.str.match("relax_rules"),
        inference_col.str.match("No_rules_relaxed") & latest_loan_dpd.ge(term_frequency+30),
        inference_col.str.match("No_rules_relaxed") & repayment_ratio.lt(repayment_ratio_threshold) & latest_loan_dpd.gt(term_frequency + dpd_allowance),
        inference_col.str.match("No_rules_relaxed") & repayment_ratio.lt(repayment_ratio_threshold) & latest_loan_dpd.le(term_frequency + dpd_allowance),
        inference_col.str.match("No_rules_relaxed") & repayment_ratio.ge(repayment_ratio_threshold) & latest_loan_dpd.ge(term_frequency + 30),
        inference_col.str.match("No_rules_relaxed") & repayment_ratio.ge(repayment_ratio_threshold) & latest_loan_dpd.le(term_frequency + dpd_allowance),
        inference_col.str.match("No_rules_relaxed") & repayment_ratio.ge(repayment_ratio_threshold) & latest_loan_dpd.ge(term_frequency + 20),
        inference_col.str.match("No_rules_relaxed") & repayment_ratio.ge(repayment_ratio_threshold) & latest_loan_dpd.ge(term_frequency + 29),
        
    ]  
    
    
    choices = [
        limit_col,
        limit_col,
        limit_col,
        zeroize,
        zeroize,
        limit_col,
        zeroize,
        limit_col,
        limit_col * 0.7,
        limit_col * 0.3,
        
    ]
    
    limit_column = np.select(conditions, choices)
    
    return limit_column


#apply the function to the df to create 21 day adjusted product limit allocation
df["adjusted_21_limit"] = limits_decrease_zeroization_21(df)

In [ ]:
def limits_decrease_zeroization_7(df):
    """
    function to adjust allocated limits in line with past repayment behavior
    
    Inputs:
    1) num of loans taken,
    2) previously allocated limit,
    3) term frequency for a loan,
    4) days past due for the most recent loan
    5) good loans repayment ratio
    6) inference variable i.e declaring whether borrower qualifies for limit stabilization OR not
    
    Output:
    adjusted limit based on rollover/default patterns of most recent loan
    """
    dpd_allowance = 10
    repayment_ratio_threshold = 0.6
    zeroize = 0
    
    #loan_count = df["loan_count"]
    loan_count = df["adjusted_loan_count"]
    limit_col = df["limit_7_day"]
    latest_loan_dpd = df["days_past_due"]
    term_frequency = df["term_frequency"]
    repayment_ratio = df["good_loans_repayment_ratio"]
    inference_col = df["inference_col"]
    
    
    conditions = [
        loan_count.isna(),
        loan_count.eq(0),
        inference_col.str.match("relax_rules"),
        inference_col.str.match("No_rules_relaxed") & latest_loan_dpd.ge(term_frequency+30),
        inference_col.str.match("No_rules_relaxed") & repayment_ratio.lt(repayment_ratio_threshold) & latest_loan_dpd.gt(term_frequency + dpd_allowance),
        inference_col.str.match("No_rules_relaxed") & repayment_ratio.lt(repayment_ratio_threshold) & latest_loan_dpd.le(term_frequency + dpd_allowance),
        inference_col.str.match("No_rules_relaxed") & repayment_ratio.ge(repayment_ratio_threshold) & latest_loan_dpd.ge(term_frequency + 30),
        inference_col.str.match("No_rules_relaxed") & repayment_ratio.ge(repayment_ratio_threshold) & latest_loan_dpd.le(term_frequency + dpd_allowance),
        inference_col.str.match("No_rules_relaxed") & repayment_ratio.ge(repayment_ratio_threshold) & latest_loan_dpd.ge(term_frequency + 20),
        inference_col.str.match("No_rules_relaxed") & repayment_ratio.ge(repayment_ratio_threshold) & latest_loan_dpd.ge(term_frequency + 29),
        
    ]  
    
    
    choices = [
        limit_col,
        limit_col,
        limit_col,
        zeroize,
        zeroize,
        limit_col,
        zeroize,
        limit_col,
        limit_col * 0.7,
        limit_col * 0.3,
        
    ]
    
    limit_column = np.select(conditions, choices)
    
    return limit_column


#apply the function to the df to create 21 day adjusted product limit allocation
df["adjusted_7_limit"] = limits_decrease_zeroization_7(df)

In [ ]:
def limits_decrease_zeroization_1(df):
    """
    function to adjust allocated limits in line with past repayment behavior
    
    Inputs:
    1) num of loans taken,
    2) previously allocated limit,
    3) term frequency for a loan,
    4) days past due for the most recent loan
    5) good loans repayment ratio
    6) inference variable i.e declaring whether borrower qualifies for limit stabilization OR not
    
    Output:
    adjusted limit based on rollover/default patterns of most recent loan
    """
    dpd_allowance = 10
    repayment_ratio_threshold = 0.6
    zeroize = 0
    
    #loan_count = df["loan_count"]
    loan_count = df["adjusted_loan_count"]
    limit_col = df["limit_1_day"]
    latest_loan_dpd = df["days_past_due"]
    term_frequency = df["term_frequency"]
    repayment_ratio = df["good_loans_repayment_ratio"]
    inference_col = df["inference_col"]
    
    
    conditions = [
        loan_count.isna(),
        loan_count.eq(0),
        inference_col.str.match("relax_rules"),
        inference_col.str.match("No_rules_relaxed") & latest_loan_dpd.ge(term_frequency+30),
        inference_col.str.match("No_rules_relaxed") & repayment_ratio.lt(repayment_ratio_threshold) & latest_loan_dpd.gt(term_frequency + dpd_allowance),
        inference_col.str.match("No_rules_relaxed") & repayment_ratio.lt(repayment_ratio_threshold) & latest_loan_dpd.le(term_frequency + dpd_allowance),
        inference_col.str.match("No_rules_relaxed") & repayment_ratio.ge(repayment_ratio_threshold) & latest_loan_dpd.ge(term_frequency + 30),
        inference_col.str.match("No_rules_relaxed") & repayment_ratio.ge(repayment_ratio_threshold) & latest_loan_dpd.le(term_frequency + dpd_allowance),
        inference_col.str.match("No_rules_relaxed") & repayment_ratio.ge(repayment_ratio_threshold) & latest_loan_dpd.ge(term_frequency + 20),
        inference_col.str.match("No_rules_relaxed") & repayment_ratio.ge(repayment_ratio_threshold) & latest_loan_dpd.ge(term_frequency + 29),
        
    ]  
    
    
    choices = [
        limit_col,
        limit_col,
        limit_col,
        zeroize,
        zeroize,
        limit_col,
        zeroize,
        limit_col,
        limit_col * 0.7,
        limit_col * 0.3,
        
    ]
    
    limit_column = np.select(conditions, choices)
    
    return limit_column


#apply the function to the df to create 21 day adjusted product limit allocation
df["adjusted_1_limit"] = limits_decrease_zeroization_1(df)

In [ ]:
df[["adjusted_21_limit","adjusted_7_limit","adjusted_1_limit"]].sum()

adjusted_21_limit   1057646877.28
adjusted_7_limit     825578343.74
adjusted_1_limit     825578343.74
dtype: float64

In [ ]:
df.shape

(99614, 61)

In [ ]:
def limit_zeroization_till_summary_21(df):
    """
    function to adjust limits based on till summaries i.e till activity and recency
    
    Inputs:
    1) recency of transactions boolean check,
    2) till consistency calculated probability,
    3) previously allocated limit,
    4) inference variable i.e declaring whether borrower qualifies for limit stabilization OR not
    
    Output:
    adjusted limits in line with till summaries i.e limits zeroized for any customer who does not meet set threshold
    """
    transaction_boolean_accepted = "Yes"
    transaction_boolean_rejected = "No"
    consistency_threshold = 0.7
    zeroize = 0
    
    transaction_boolean_col = df["transacted_last_5_days"]
    consistency_col = df["page_active_days"]
    limit_col = df["adjusted_21_limit"]
    inference_col = df["inference_col"]
    
    conditions = [
        inference_col.str.match("relax_rules"),
        inference_col.str.match("No_rules_relaxed") & transaction_boolean_col.str.contains(transaction_boolean_accepted) & consistency_col.ge(consistency_threshold),
        inference_col.str.match("No_rules_relaxed") & transaction_boolean_col.str.contains(transaction_boolean_accepted) & consistency_col.lt(consistency_threshold),
        inference_col.str.match("No_rules_relaxed") & transaction_boolean_col.str.contains(transaction_boolean_rejected) & consistency_col.ge(consistency_threshold),
        inference_col.str.match("No_rules_relaxed") & transaction_boolean_col.str.contains(transaction_boolean_rejected) & consistency_col.lt(consistency_threshold),
        
        
    ]
    
    choices = [
        limit_col,
        limit_col,
        zeroize,
        zeroize,
        zeroize,
    ]
    
    limit_column = np.select(conditions, choices)
    
    return limit_column

#apply the function to the df to create 21 day adjusted product limit allocation
df["adjusted_21_limit"] = limit_zeroization_till_summary_21(df)

In [ ]:
def limit_zeroization_till_summary_7(df):
    """
    function to adjust limits based on till summaries i.e till activity and recency
    
    Inputs:
    1) recency of transactions boolean check,
    2) till consistency calculated probability,
    3) previously allocated limit,
    4) inference variable i.e declaring whether borrower qualifies for limit stabilization OR not
    
    Output:
    adjusted limits in line with till summaries i.e limits zeroized for any customer who does not meet set threshold
    """
    
    transaction_boolean_accepted = "Yes"
    transaction_boolean_rejected = "No"
    consistency_threshold = 0.7
    zeroize = 0
    
    transaction_boolean_col = df["transacted_last_5_days"]
    consistency_col = df["page_active_days"]
    limit_col = df["adjusted_7_limit"]
    inference_col = df["inference_col"]
    
    conditions = [
        inference_col.str.match("relax_rules"),
        inference_col.str.match("No_rules_relaxed") & transaction_boolean_col.str.contains(transaction_boolean_accepted) & consistency_col.ge(consistency_threshold),
        inference_col.str.match("No_rules_relaxed") & transaction_boolean_col.str.contains(transaction_boolean_accepted) & consistency_col.lt(consistency_threshold),
        inference_col.str.match("No_rules_relaxed") & transaction_boolean_col.str.contains(transaction_boolean_rejected) & consistency_col.ge(consistency_threshold),
        inference_col.str.match("No_rules_relaxed") & transaction_boolean_col.str.contains(transaction_boolean_rejected) & consistency_col.lt(consistency_threshold),
        
        
    ]
    
    choices = [
        limit_col,
        limit_col,
        zeroize,
        zeroize,
        zeroize,
    ]
    
    limit_column = np.select(conditions, choices)
    
    return limit_column

#apply the function to the df to create 21 day adjusted product limit allocation
df["adjusted_7_limit"] = limit_zeroization_till_summary_7(df)

In [ ]:
def limit_zeroization_till_summary_1(df):
    """
    function to adjust limits based on till summaries i.e till activity and recency
    
    Inputs:
    1) recency of transactions boolean check,
    2) till consistency calculated probability,
    3) previously allocated limit,
    4) inference variable i.e declaring whether borrower qualifies for limit stabilization OR not
    
    Output:
    adjusted limits in line with till summaries i.e limits zeroized for any customer who does not meet set threshold
    """
    transaction_boolean_accepted = "Yes"
    transaction_boolean_rejected = "No"
    consistency_threshold = 0.7
    zeroize = 0
    
    transaction_boolean_col = df["transacted_last_5_days"]
    consistency_col = df["page_active_days"]
    limit_col = df["adjusted_1_limit"]
    inference_col = df["inference_col"]
    
    conditions = [
        inference_col.str.match("relax_rules"),
        inference_col.str.match("No_rules_relaxed") & transaction_boolean_col.str.contains(transaction_boolean_accepted) & consistency_col.ge(consistency_threshold),
        inference_col.str.match("No_rules_relaxed") & transaction_boolean_col.str.contains(transaction_boolean_accepted) & consistency_col.lt(consistency_threshold),
        inference_col.str.match("No_rules_relaxed") & transaction_boolean_col.str.contains(transaction_boolean_rejected) & consistency_col.ge(consistency_threshold),
        inference_col.str.match("No_rules_relaxed") & transaction_boolean_col.str.contains(transaction_boolean_rejected) & consistency_col.lt(consistency_threshold),
        
        
    ]
    
    choices = [
        limit_col,
        limit_col,
        zeroize,
        zeroize,
        zeroize,
    ]
    
    limit_column = np.select(conditions, choices)
    
    return limit_column

#apply the function to the df to create 21 day adjusted product limit allocation
df["adjusted_1_limit"] = limit_zeroization_till_summary_1(df)

In [ ]:
df[["adjusted_21_limit","adjusted_7_limit","adjusted_1_limit"]].sum()

adjusted_21_limit   1047561606.73
adjusted_7_limit     804174835.66
adjusted_1_limit     804174835.66
dtype: float64

In [ ]:
def adjust_limits_to_loan_bands_21(df):
    """
    function to limit loan limits based on loan bands
    
    Inputs:
    1) loan count of a borrower, 
    2) allocated limit
    
    Output:
    adjusted limit based on loan band caps
    """
    #loan band matrix
    no_loans = 0
    
    
    #loan_count = df["loan_count"]
    loan_count = df["adjusted_loan_count"]
    limit_col = df["adjusted_21_limit"]

    conditions = [
        loan_count.isna(),
        loan_count.isna(),
        loan_count.eq(0),
        loan_count.eq(0),
        loan_count.gt(0)
    ]
    
    
    choices = [
        no_loans,
        no_loans,
        no_loans,
        no_loans,
        limit_col
    ]
    
    limit_column = np.select(conditions, choices)
    
    return limit_column


df["adjusted_21_limit"] = adjust_limits_to_loan_bands_21(df)

In [ ]:
df[["adjusted_21_limit","adjusted_7_limit","adjusted_1_limit"]].sum()

adjusted_21_limit   758137781.84
adjusted_7_limit    804174835.66
adjusted_1_limit    804174835.66
dtype: float64

In [ ]:
# defining a function for the limit caps
def calc_new_final_21_limit(df):
    adjusted_21_limit = df['adjusted_21_limit']
  
    if adjusted_21_limit < 1000:
        return 0
    elif adjusted_21_limit > 200000:
        return 200000
    else:
        return adjusted_21_limit


def calc_new_final_7_limit(df):
    adjusted_7_limit = df['adjusted_7_limit']
  
    if adjusted_7_limit < 1000:
        return 0
    elif adjusted_7_limit > 200000:
        return 200000
    else:
        return adjusted_7_limit


def calc_new_final_1_limit(df):
    adjusted_1_limit = df['adjusted_1_limit']
  
    if adjusted_1_limit < 200:
        return 0
    elif adjusted_1_limit > 200000:
        return 200000
    else:
        return adjusted_1_limit

In [ ]:
# adding new_final_21_limit, new_final_7_limit and new_final_1_limit columns to the dataframe by applying the functions to the dataframe
df['final_21_limit'] = df.apply(lambda x: calc_new_final_21_limit(x), axis = 1)
df['final_7_limit'] = df.apply(lambda x: calc_new_final_7_limit(x), axis = 1)
df['final_1_limit'] = df.apply(lambda x: calc_new_final_1_limit(x), axis = 1)

df.head(3)

,mobile_number,store_number,approx_30_days_trx_val,most_recent_trx_date_past_30_days,last_trx_date,expected_trx_days,actual_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,IDM_recommendation,IDM_limit,is_iprs_validated,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,IDM_factor_21,IDM_factor_7,IDM_factor_1,trading_consistency_bands,loan_count_bands,new_limit_factor_21,new_limit_factor_7,new_limit_factor_1,weight_good_loans_repayment_ratio,weight_consistency,weight_recency,risk_rules_factor,ultimate_factor_21,ultimate_factor_7,ultimate_factor_1,limit_21_day,limit_7_day,limit_1_day,adjusted_21_limit,adjusted_7_limit,adjusted_1_limit,final_21_limit,final_7_limit,final_1_limit
92174,254722978849,822369,156.67,2022-06-18,2022-06-18,1,1,1.00,relax_rules,88,No,0.00,22128407,Approve,0.00,True,NaN,4.00,600.00,7.00,5000.00,5000.00,2017-12-10,2017-12-17,2017-12-10,2017-12-17,-7.00,1.00,closed_early_repayment,NaN,5000.00,2017-12-10,4.00,1.00,1738.00,1,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 7,Band 3,0.30,0.17,0.17,1.00,1.00,0,0.75,0.22,0.13,0.13,35.25,20.56,20.56,0.00,20.56,20.56,0.00,0.00,0.00
84778,254723173634,786152,2050.00,2022-06-14,2022-09-08,87,12,0.14,No_rules_relaxed,6,No,0.00,494024,Approve,0.00,True,NaN,1.00,601.00,30.00,12698.00,9206.00,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.00,1.00,written-off_default,NaN,12698.00,2017-12-18,0.00,0.00,1730.00,0,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 1,Band 2,0.00,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1380,254723916436,165978,810233.88,2022-06-13,2022-09-12,92,91,0.99,No_rules_relaxed,2,Yes,1.00,20417564,Approve,0.00,True,NaN,1.00,601.00,30.00,20000.00,12800.00,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.00,1.00,written-off_default,NaN,20000.00,2017-12-18,0.00,0.00,1730.00,0,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 7,Band 2,0.25,0.15,0.15,0.00,1.00,2,0.75,0.19,0.11,0.11,151918.85,91151.31,91151.31,0.00,91151.31,91151.31,0.00,91151.31,91151.31


In [ ]:
#convert store_number col to string type
df['final_21_limit'] = df['final_21_limit'].fillna(0)
df['final_7_limit'] = df['final_7_limit'].fillna(0)
df['final_1_limit'] = df['final_1_limit'].fillna(0)

In [ ]:
#apply ceiling 100 to final limit columns to adjust limits to the nearest 100 value 

df["final_21_limit"] = (np.ceil(df["final_21_limit"] / 100) * 100).astype(int)
df["final_7_limit"] = (np.ceil(df["final_7_limit"] / 100) * 100).astype(int)
df["final_1_limit"] = (np.ceil(df["final_1_limit"] / 100) * 100).astype(int)

In [ ]:
#drop duplicates based on store_number/national id

df = df.drop_duplicates(subset="store_number", keep="first")

In [ ]:
df[["final_21_limit","final_7_limit","final_1_limit"]].sum()

final_21_limit    532659400
final_7_limit     697185700
final_1_limit     700454100
dtype: int64

In [ ]:
df.shape

(99614, 64)

In [ ]:
df['IDM_recommendation'].value_counts()

Reject     75143
Approve    24471
Name: IDM_recommendation, dtype: int64

In [ ]:
#check against MFS list of defaults

#mfs_default_path = "C:\\Project_summaries\\Bloom\\Bloom all_loans\\20220721\\MFS_crosscheck\\"

mfs_defaulters_list = pd.read_excel("/content/mfs_defaulters_list.xlsx")

In [ ]:
mfs_defaulters_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8050 entries, 0 to 8049
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   store_number  8050 non-null   int64 
 1   national_id   8050 non-null   object
dtypes: int64(1), object(1)
memory usage: 125.9+ KB


In [ ]:
mfs_defaulters_list['store_number'] = mfs_defaulters_list['store_number'].astype(str)

mfs_defaulters_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8050 entries, 0 to 8049
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   store_number  8050 non-null   object
 1   national_id   8050 non-null   object
dtypes: object(2)
memory usage: 125.9+ KB


In [ ]:
#get list of defaulters

mfs_defaulters_list = list(mfs_defaulters_list["store_number"].unique())

In [ ]:
#assign a zero limit to every national id that matches the mfs defaulters list

df.loc[(df["store_number"].isin(mfs_defaulters_list))&(df["final_21_limit"]>0), "final_21_limit"] = 0
df.loc[(df["store_number"].isin(mfs_defaulters_list))&(df["final_7_limit"]>0), "final_7_limit"] = 0
df.loc[(df["store_number"].isin(mfs_defaulters_list))&(df["final_1_limit"]>0), "final_1_limit"] = 0

In [ ]:
df[["final_21_limit","final_7_limit","final_1_limit"]].sum()

final_21_limit    532659400
final_7_limit     693192300
final_1_limit     696427800
dtype: int64

In [ ]:
#exempt totally new customers who have not taken any loan but have failed our rules

excluded_first_time_df = df[((df["loan_count"]==0)&(df["final_7_limit"]==0)&(df["final_1_limit"]==0))]
df = df[~((df["loan_count"]==0)&(df["final_7_limit"]==0)&(df["final_1_limit"]==0))]

In [ ]:
df.shape

(56820, 64)

In [ ]:
df[["final_21_limit","final_7_limit","final_1_limit"]].sum()

final_21_limit    532659400
final_7_limit     693192300
final_1_limit     696427800
dtype: int64

In [ ]:
df['IDM_recommendation'].value_counts()

Reject     40769
Approve    16051
Name: IDM_recommendation, dtype: int64

In [ ]:
def calc_blacklist_flag(df):
    
    days_past_due = df["days_past_due"]
    bloom_version = df["bloom_version"]
    loan_status = df["loan_status"]
    
    if (bloom_version == 1 and loan_status == 300) or days_past_due >= 60:
        return 1
    else:
        return 0

In [ ]:
df['blacklist_flag'] = df.apply(lambda x: calc_blacklist_flag(x), axis = 1)

df.head(3)

,mobile_number,store_number,approx_30_days_trx_val,most_recent_trx_date_past_30_days,last_trx_date,expected_trx_days,actual_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,IDM_recommendation,IDM_limit,is_iprs_validated,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,IDM_factor_21,IDM_factor_7,IDM_factor_1,trading_consistency_bands,loan_count_bands,new_limit_factor_21,new_limit_factor_7,new_limit_factor_1,weight_good_loans_repayment_ratio,weight_consistency,weight_recency,risk_rules_factor,ultimate_factor_21,ultimate_factor_7,ultimate_factor_1,limit_21_day,limit_7_day,limit_1_day,adjusted_21_limit,adjusted_7_limit,adjusted_1_limit,final_21_limit,final_7_limit,final_1_limit,blacklist_flag
92174,254722978849,822369,156.67,2022-06-18,2022-06-18,1,1,1.00,relax_rules,88,No,0.00,22128407,Approve,0.00,True,NaN,4.00,600.00,7.00,5000.00,5000.00,2017-12-10,2017-12-17,2017-12-10,2017-12-17,-7.00,1.00,closed_early_repayment,NaN,5000.00,2017-12-10,4.00,1.00,1738.00,1,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 7,Band 3,0.30,0.17,0.17,1.00,1.00,0,0.75,0.22,0.13,0.13,35.25,20.56,20.56,0.00,20.56,20.56,0,0,0,0
84778,254723173634,786152,2050.00,2022-06-14,2022-09-08,87,12,0.14,No_rules_relaxed,6,No,0.00,494024,Approve,0.00,True,NaN,1.00,601.00,30.00,12698.00,9206.00,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.00,1.00,written-off_default,NaN,12698.00,2017-12-18,0.00,0.00,1730.00,0,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 1,Band 2,0.00,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,1
1380,254723916436,165978,810233.88,2022-06-13,2022-09-12,92,91,0.99,No_rules_relaxed,2,Yes,1.00,20417564,Approve,0.00,True,NaN,1.00,601.00,30.00,20000.00,12800.00,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.00,1.00,written-off_default,NaN,20000.00,2017-12-18,0.00,0.00,1730.00,0,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 7,Band 2,0.25,0.15,0.15,0.00,1.00,2,0.75,0.19,0.11,0.11,151918.85,91151.31,91151.31,0.00,91151.31,91151.31,0,91200,91200,1


In [ ]:
df['blacklist_flag'].value_counts()

0    39098
1    17722
Name: blacklist_flag, dtype: int64

In [ ]:
df[df['store_number'] == 7926899]

,mobile_number,store_number,approx_30_days_trx_val,most_recent_trx_date_past_30_days,last_trx_date,expected_trx_days,actual_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,IDM_recommendation,IDM_limit,is_iprs_validated,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,IDM_factor_21,IDM_factor_7,IDM_factor_1,trading_consistency_bands,loan_count_bands,new_limit_factor_21,new_limit_factor_7,new_limit_factor_1,weight_good_loans_repayment_ratio,weight_consistency,weight_recency,risk_rules_factor,ultimate_factor_21,ultimate_factor_7,ultimate_factor_1,limit_21_day,limit_7_day,limit_1_day,adjusted_21_limit,adjusted_7_limit,adjusted_1_limit,final_21_limit,final_7_limit,final_1_limit,blacklist_flag


In [ ]:
#remove any customers not blacklist and limits allocated across board is zero

#df = df.loc[~((df["blacklist_flag"]==0)&(df["final_7_limit"]==0)&(df["final_7_limit"]==0)&(df["final_1_limit"]==0))]

In [ ]:
#make all column headers to be in small letters & trim out any trailing or white spaces

df.columns = df.columns.str.lower()

In [ ]:
df['blacklist_flag'].value_counts()

0    39098
1    17722
Name: blacklist_flag, dtype: int64

In [ ]:
#add limit factor column

def declare_limit_factors_21(df):
    """
    function to declare limit factors used to allocate limits based on defined conditions i.e IDM recommendation
    
    Inputs:
    1) IDM recommendation column i.e IDM Approve VS IDM Reject
    
    Output:
    assigned limit factors used to allocate limits based on defined conditions i.e IDM recommendation
    """
    
    idm_recommendation = df["idm_recommendation"]
    
    limit_21_approve = 0.5
    limit_21_reject = 0.35
   
    conditions = [
        idm_recommendation.str.match("Approve"),
        idm_recommendation.str.match("Reject"),              
    ]
    
    choices = [
        limit_21_approve,
        limit_21_reject,
    ]
    
    limit_factor_21 = np.select(conditions, choices)
    
    return limit_factor_21

df["limit_factor_21"] = declare_limit_factors_21(df)

In [ ]:
#add limit factor column

def declare_limit_factors_7(df):
    """
    function to declare limit factors used to allocate limits based on defined conditions i.e IDM recommendation
    
    Inputs:
    1) IDM recommendation column i.e IDM Approve VS IDM Reject
    
    Output:
    assigned limit factors used to allocate limits based on defined conditions i.e IDM recommendation
    """
    idm_recommendation = df["idm_recommendation"]
    
    limit_7_approve = 0.17
    limit_7_reject = 0.12
   
    conditions = [
        idm_recommendation.str.match("Approve"),
        idm_recommendation.str.match("Reject"),              
    ]
    
    choices = [
        limit_7_approve,
        limit_7_reject,
    ]
    
    limit_factor_7 = np.select(conditions, choices)
    
    return limit_factor_7

df["limit_factor_7"] = declare_limit_factors_7(df)

In [ ]:
#add limit factor column

def declare_limit_factors_1(df):
    """
    function to declare limit factors used to allocate limits based on defined conditions i.e IDM recommendation
    
    Inputs:
    1) IDM recommendation column i.e IDM Approve VS IDM Reject
    
    Output:
    assigned limit factors used to allocate limits based on defined conditions i.e IDM recommendation
    """
    idm_recommendation = df["idm_recommendation"]
    
    limit_1_approve = 0.17
    limit_1_reject = 0.12
   
    conditions = [
        idm_recommendation.str.match("Approve"),
        idm_recommendation.str.match("Reject"),              
    ]
    
    choices = [
        limit_1_approve,
        limit_1_reject,
    ]
    
    limit_factor_1 = np.select(conditions, choices)
    
    return limit_factor_1

df["limit_factor_1"] = declare_limit_factors_1(df)

In [ ]:
df.head(3)

,mobile_number,store_number,approx_30_days_trx_val,most_recent_trx_date_past_30_days,last_trx_date,expected_trx_days,actual_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,idm_recommendation,idm_limit,is_iprs_validated,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,idm_factor_21,idm_factor_7,idm_factor_1,trading_consistency_bands,loan_count_bands,new_limit_factor_21,new_limit_factor_7,new_limit_factor_1,weight_good_loans_repayment_ratio,weight_consistency,weight_recency,risk_rules_factor,ultimate_factor_21,ultimate_factor_7,ultimate_factor_1,limit_21_day,limit_7_day,limit_1_day,adjusted_21_limit,adjusted_7_limit,adjusted_1_limit,final_21_limit,final_7_limit,final_1_limit,blacklist_flag
92174,254722978849,822369,156.67,2022-06-18,2022-06-18,1,1,1.00,relax_rules,88,No,0.00,22128407,Approve,0.00,True,NaN,4.00,600.00,7.00,5000.00,5000.00,2017-12-10,2017-12-17,2017-12-10,2017-12-17,-7.00,1.00,closed_early_repayment,NaN,5000.00,2017-12-10,4.00,1.00,1738.00,1,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 7,Band 3,0.30,0.17,0.17,1.00,1.00,0,0.75,0.22,0.13,0.13,35.25,20.56,20.56,0.00,20.56,20.56,0,0,0,0
84778,254723173634,786152,2050.00,2022-06-14,2022-09-08,87,12,0.14,No_rules_relaxed,6,No,0.00,494024,Approve,0.00,True,NaN,1.00,601.00,30.00,12698.00,9206.00,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.00,1.00,written-off_default,NaN,12698.00,2017-12-18,0.00,0.00,1730.00,0,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 1,Band 2,0.00,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,1
1380,254723916436,165978,810233.88,2022-06-13,2022-09-12,92,91,0.99,No_rules_relaxed,2,Yes,1.00,20417564,Approve,0.00,True,NaN,1.00,601.00,30.00,20000.00,12800.00,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.00,1.00,written-off_default,NaN,20000.00,2017-12-18,0.00,0.00,1730.00,0,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 7,Band 2,0.25,0.15,0.15,0.00,1.00,2,0.75,0.19,0.11,0.11,151918.85,91151.31,91151.31,0.00,91151.31,91151.31,0,91200,91200,1


In [ ]:
df.shape

(56820, 65)

In [ ]:
df[df['store_number'] == 7926899]

,mobile_number,store_number,approx_30_days_trx_val,most_recent_trx_date_past_30_days,last_trx_date,expected_trx_days,actual_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,idm_recommendation,idm_limit,is_iprs_validated,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,idm_factor_21,idm_factor_7,idm_factor_1,trading_consistency_bands,loan_count_bands,new_limit_factor_21,new_limit_factor_7,new_limit_factor_1,weight_good_loans_repayment_ratio,weight_consistency,weight_recency,risk_rules_factor,ultimate_factor_21,ultimate_factor_7,ultimate_factor_1,limit_21_day,limit_7_day,limit_1_day,adjusted_21_limit,adjusted_7_limit,adjusted_1_limit,final_21_limit,final_7_limit,final_1_limit,blacklist_flag


In [ ]:
df['idm_recommendation'].value_counts()

Reject     40769
Approve    16051
Name: idm_recommendation, dtype: int64

In [ ]:
df[["final_21_limit","final_7_limit","final_1_limit"]].sum()

final_21_limit    532659400
final_7_limit     693192300
final_1_limit     696427800
dtype: int64

#### no new 21-day customers

In [ ]:
host = '157.245.248.249'
port = int(5432)
dbname = 'ubuntu'
user = 'jacklinengenia'
password = 'x3MX&8#!'


def get_query_results_postgres():
    with psycopg2.connect(host = host,
                          port = port,
                          database = dbname,
                          user = user,
                          password = password) as conn:
        sql = "SELECT * FROM bloomlive.scoring_results"
        df = pd.read_sql(sql, conn)
        
    return df

    conn.close()
    
if __name__ == "__main__":
    get_query_results_postgres() 

In [ ]:
scoring_results = get_query_results_postgres()

scoring_results.head(3)

In [ ]:
scoring_results_sum = scoring_results.groupby(['store_number'], as_index=False)['final_21_limit'].sum()
scoring_results_sum = scoring_results_sum.rename(columns={'final_21_limit':'total_final_21_limit'})

scoring_results_sum.head(10)

,store_number,total_final_21_limit
0,000030,0.00
1,001587,0.00
2,001591,0.00
3,001611,0.00
4,001713,0.00
5,001715,0.00
6,001717,0.00
7,001725,0.00
8,001731,0.00
9,001735,0.00


In [ ]:
df = pd.merge(df, scoring_results_sum, on = 'store_number', how = 'left')

df.head(3)

,mobile_number,store_number,approx_30_days_trx_val,most_recent_trx_date_past_30_days,last_trx_date,expected_trx_days,actual_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,idm_recommendation,idm_limit,is_iprs_validated,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,idm_factor_21,idm_factor_7,idm_factor_1,trading_consistency_bands,loan_count_bands,new_limit_factor_21,new_limit_factor_7,new_limit_factor_1,weight_good_loans_repayment_ratio,weight_consistency,weight_recency,risk_rules_factor,ultimate_factor_21,ultimate_factor_7,ultimate_factor_1,limit_21_day,limit_7_day,limit_1_day,adjusted_21_limit,adjusted_7_limit,adjusted_1_limit,final_21_limit,final_7_limit,final_1_limit,blacklist_flag,total_final_21_limit
0,254722978849,822369,156.67,2022-06-18,2022-06-18,1,1,1.00,relax_rules,88,No,0.00,22128407,Approve,0.00,True,NaN,4.00,600.00,7.00,5000.00,5000.00,2017-12-10,2017-12-17,2017-12-10,2017-12-17,-7.00,1.00,closed_early_repayment,NaN,5000.00,2017-12-10,4.00,1.00,1738.00,1,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 7,Band 3,0.30,0.17,0.17,1.00,1.00,0,0.75,0.22,0.13,0.13,35.25,20.56,20.56,0.00,20.56,20.56,0,0,0,0,0.00
1,254723173634,786152,2050.00,2022-06-14,2022-09-08,87,12,0.14,No_rules_relaxed,6,No,0.00,494024,Approve,0.00,True,NaN,1.00,601.00,30.00,12698.00,9206.00,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.00,1.00,written-off_default,NaN,12698.00,2017-12-18,0.00,0.00,1730.00,0,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 1,Band 2,0.00,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,1,0.00
2,254723916436,165978,810233.88,2022-06-13,2022-09-12,92,91,0.99,No_rules_relaxed,2,Yes,1.00,20417564,Approve,0.00,True,NaN,1.00,601.00,30.00,20000.00,12800.00,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.00,1.00,written-off_default,NaN,20000.00,2017-12-18,0.00,0.00,1730.00,0,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 7,Band 2,0.25,0.15,0.15,0.00,1.00,2,0.75,0.19,0.11,0.11,151918.85,91151.31,91151.31,0.00,91151.31,91151.31,0,91200,91200,1,304500.00


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56820 entries, 0 to 56819
Data columns (total 66 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   mobile_number                      44567 non-null  object        
 1   store_number                       56820 non-null  object        
 2   approx_30_days_trx_val             56820 non-null  float64       
 3   most_recent_trx_date_past_30_days  56820 non-null  datetime64[ns]
 4   last_trx_date                      56820 non-null  datetime64[ns]
 5   expected_trx_days                  56820 non-null  int64         
 6   actual_trx_days                    56820 non-null  int64         
 7   page_active_days                   56820 non-null  float64       
 8   inference_col                      56820 non-null  object        
 9   days_since_last_trx                56820 non-null  int64         
 10  transacted_last_5_days            

In [ ]:
cols_fillna = ["total_final_21_limit"]
# replace 'NaN' with zero in these columns
for col in cols_fillna:
    df[col].fillna(0,inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56820 entries, 0 to 56819
Data columns (total 66 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   mobile_number                      44567 non-null  object        
 1   store_number                       56820 non-null  object        
 2   approx_30_days_trx_val             56820 non-null  float64       
 3   most_recent_trx_date_past_30_days  56820 non-null  datetime64[ns]
 4   last_trx_date                      56820 non-null  datetime64[ns]
 5   expected_trx_days                  56820 non-null  int64         
 6   actual_trx_days                    56820 non-null  int64         
 7   page_active_days                   56820 non-null  float64       
 8   inference_col                      56820 non-null  object        
 9   days_since_last_trx                56820 non-null  int64         
 10  transacted_last_5_days            

In [ ]:
def calc_final_21_limit(df):
    total_final_21_limit = df['total_final_21_limit']
    final_21_limit = df['final_21_limit']

    if total_final_21_limit == 0:
        return 0
    elif total_final_21_limit > 0:
        return final_21_limit

In [ ]:
df['final_21_limit'] = df.apply(lambda x: calc_final_21_limit(x), axis = 1)

In [ ]:
df[["final_21_limit","final_7_limit","final_1_limit"]].sum()

final_21_limit    451392200
final_7_limit     693192300
final_1_limit     696427800
dtype: int64

#### cap limits by up to 25% of previous limit

In [ ]:
#pull data from db so as to cap limits by up to 25% of previous limit

# Connection parameters
param_dic = {
    "host"      : "157.245.248.249",
    "database"  : "ubuntu",
    "user"      : "jacklinengenia",
    "password"  : "x3MX&8#!"
}

def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

In [ ]:
def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # Convert the list of tuples in to a pandas dataframe
    previous_results_df = pd.DataFrame(tupples, columns=column_names)
    return previous_results_df

In [ ]:
# Connect to the database and load the scoring data bloomlive table
conn = connect(param_dic)

columns = "store_number, previous_21_limit, previous_7_limit, previous_1_limit"

column_names = columns.strip().split(",")
# Execute the "SELECT cols" query
previous_results_df = postgresql_to_dataframe(conn,
                             "select \
                             store_number, final_21_limit as previous_21_limit,final_7_limit as previous_7_limit,final_1_limit as previous_1_limit\
                             from bloomlive.scoring_results\
                             where model_version='2022-004[2022-05-14, 2022-09-02]'",
                             column_names)

#drop duplicates
previous_results_df = previous_results_df.drop_duplicates(subset="store_number", keep="first")

Connecting to the PostgreSQL database...
Connection successful


In [ ]:
previous_results_df.head(3)

,store_number,previous_21_limit,previous_7_limit,previous_1_limit
0,668934,0,0,0
1,963370,0,0,0
2,866692,0,0,0


In [ ]:
#convert store number col to str type

df["store_number"] = df["store_number"].astype("str")

In [ ]:
#left merge with main df to assess for previous limit allocation

df = pd.merge(df, previous_results_df, how="left", on="store_number")

In [ ]:
#strip out white spaces from column names

df = df.rename(columns=lambda x: x.strip())

In [ ]:
df["previous_21_limit"].unique()

array([Decimal('0'), Decimal('30400'), Decimal('200000'),
       Decimal('35300'), Decimal('2500'), Decimal('45200'),
       Decimal('127400'), Decimal('1600'), Decimal('1500'),
       Decimal('12800'), Decimal('32300'), Decimal('94000'),
       Decimal('10600'), Decimal('25700'), Decimal('22300'),
       Decimal('1000'), Decimal('25000'), Decimal('43300'),
       Decimal('45100'), Decimal('26500'), Decimal('22000'),
       Decimal('70700'), Decimal('8700'), Decimal('30700'),
       Decimal('6800'), Decimal('27000'), Decimal('18200'),
       Decimal('26900'), Decimal('165000'), Decimal('26000'),
       Decimal('114200'), Decimal('44700'), Decimal('42200'),
       Decimal('2200'), Decimal('18900'), Decimal('70800'),
       Decimal('39300'), Decimal('52400'), Decimal('6500'),
       Decimal('24800'), Decimal('59400'), Decimal('31800'),
       Decimal('5400'), Decimal('190200'), Decimal('28000'),
       Decimal('1400'), Decimal('3500'), Decimal('105300'),
       Decimal('10900'), Decimal(

In [ ]:
#convert object type columns to int

#df.loc[df["previous_21_limit"].notnull()].astype("float")
df["previous_21_limit"] = pd.to_numeric(df["previous_21_limit"], errors='coerce')
df["previous_7_limit"] = pd.to_numeric(df["previous_7_limit"], errors='coerce')
df["previous_1_limit"] = pd.to_numeric(df["previous_1_limit"], errors='coerce')

In [ ]:
#fillna previous_limit column

cols_fillna = ["previous_21_limit"]
# replace 'NaN' with zero in these columns
for col in cols_fillna:
    df[col].fillna(0,inplace=True)

cols_fillna = ["previous_7_limit"]
# replace 'NaN' with zero in these columns
for col in cols_fillna:
    df[col].fillna(0,inplace=True)

cols_fillna = ["previous_1_limit"]
# replace 'NaN' with zero in these columns
for col in cols_fillna:
    df[col].fillna(0,inplace=True)

In [ ]:
def calc_final_21_limit(df):

  final_21_limit = df['final_21_limit']
  previous_21_limit = df['previous_21_limit']

  if final_21_limit == 0:
    return final_21_limit
  elif previous_21_limit == 0:
    return final_21_limit
  elif final_21_limit >= (previous_21_limit * 0.85) or final_21_limit <= (previous_21_limit * 1.15):
    return previous_21_limit
  elif final_21_limit >= (previous_21_limit * 1.5):
    return previous_21_limit * 1.5
  else:
    return final_21_limit


def calc_final_7_limit(df):

  final_7_limit = df['final_7_limit']
  previous_7_limit = df['previous_7_limit']

  if final_7_limit == 0:
    return final_7_limit
  elif previous_7_limit == 0:
    return final_7_limit
  elif final_7_limit >= (previous_7_limit * 0.85) or final_7_limit <= (previous_7_limit * 1.15):
    return previous_7_limit
  elif final_7_limit >= (previous_7_limit * 1.5):
    return previous_7_limit * 1.5
  else:
    return final_7_limit


def calc_final_1_limit(df):

  final_1_limit = df['final_1_limit']
  previous_1_limit = df['previous_1_limit']

  if final_1_limit == 0:
    return final_1_limit
  elif previous_1_limit == 0:
    return final_1_limit
  elif final_1_limit >= (previous_1_limit * 0.85) or final_1_limit <= (previous_1_limit * 1.15):
    return previous_1_limit
  elif final_1_limit >= (previous_1_limit * 1.5):
    return previous_1_limit * 1.5
  else:
    return final_1_limit


In [ ]:
df['final_21_limit'] = df.apply(lambda x: calc_final_21_limit(x), axis = 1)
df['final_7_limit'] = df.apply(lambda x: calc_final_7_limit(x), axis = 1)
df['final_1_limit'] = df.apply(lambda x: calc_final_1_limit(x), axis = 1)

In [ ]:
df[["final_21_limit","final_7_limit","final_1_limit"]].sum()

final_21_limit   147158600.00
final_7_limit    372148600.00
final_1_limit    373431100.00
dtype: float64

#### limits adjustments to merchants with DPD 30 plus

In [ ]:
# Limit adjustment rule 1: Merchants that have DPD 30 plus on loans clients get a 50% limit reduction across all limits (1-day, 7-day and 21-day, subject to product minimum rules) 
def calc_final_21_limit(df):
    
    days_past_due = df["days_past_due"]
    final_21_limit = df["final_21_limit"]
    
    if days_past_due >= 30:
        return 0.5 * final_21_limit
    else:
        return final_21_limit
    
    
def calc_final_7_limit(df):
    
    days_past_due = df["days_past_due"]
    final_7_limit = df["final_7_limit"]
    
    if days_past_due >= 30:
        return 0.5 * final_7_limit
    else:
        return final_7_limit    
    
    
def calc_final_1_limit(df):
    
    days_past_due = df["days_past_due"]
    final_1_limit = df["final_1_limit"]
    
    if days_past_due >= 30:
        return 0.5 * final_1_limit
    else:
        return final_1_limit  

In [ ]:
df['final_21_limit'] = df.apply(lambda x: calc_final_21_limit(x), axis = 1)
df['final_7_limit'] = df.apply(lambda x: calc_final_7_limit(x), axis = 1)
df['final_1_limit'] = df.apply(lambda x: calc_final_1_limit(x), axis = 1)

In [ ]:
df[["final_21_limit","final_7_limit","final_1_limit"]].sum()

final_21_limit   147158600.00
final_7_limit    371856750.00
final_1_limit    373136800.00
dtype: float64

In [ ]:
#apply ceiling 100 to final limit columns to adjust limits to the nearest 100 value 

df["final_21_limit"] = (np.ceil(df["final_21_limit"] / 100) * 100).astype(int)
df["final_7_limit"] = (np.ceil(df["final_7_limit"] / 100) * 100).astype(int)
df["final_1_limit"] = (np.ceil(df["final_1_limit"] / 100) * 100).astype(int)

In [ ]:
df[["final_21_limit","final_7_limit","final_1_limit"]].sum()

final_21_limit    147158600
final_7_limit     371857500
final_1_limit     373137700
dtype: int64

In [ ]:
# defining a function for the limit caps
def calc_final_21_limit(df):
    final_21_limit = df['final_21_limit']
  
    if final_21_limit < 1000:
        return 0
    elif final_21_limit > 200000:
        return 200000
    else:
        return final_21_limit


def calc_final_7_limit(df):
    final_7_limit = df['final_7_limit']
  
    if final_7_limit < 1000:
        return 0
    elif final_7_limit > 200000:
        return 200000
    else:
        return final_7_limit


def calc_final_1_limit(df):
    final_1_limit = df['final_1_limit']
  
    if final_1_limit < 200:
        return 0
    elif final_1_limit > 200000:
        return 200000
    else:
        return final_1_limit

In [ ]:
df['final_21_limit'] = df.apply(lambda x: calc_final_21_limit(x), axis = 1)
df['final_7_limit'] = df.apply(lambda x: calc_final_7_limit(x), axis = 1)
df['final_1_limit'] = df.apply(lambda x: calc_final_1_limit(x), axis = 1)

In [ ]:
df[["final_21_limit","final_7_limit","final_1_limit"]].sum()

final_21_limit    147158600
final_7_limit     371856300
final_1_limit     373137700
dtype: int64

###==> resume logic

In [ ]:
# removing the test accounts from dataframe
removed_lst = ['573691', '219091', '884766', '735346', '5009587']
df = df[~df['store_number'].isin(removed_lst)]

In [ ]:
df[df['store_number'] == 7926899]

,mobile_number,store_number,approx_30_days_trx_val,most_recent_trx_date_past_30_days,last_trx_date,expected_trx_days,actual_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,idm_recommendation,idm_limit,is_iprs_validated,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,idm_factor_21,idm_factor_7,idm_factor_1,trading_consistency_bands,loan_count_bands,new_limit_factor_21,new_limit_factor_7,new_limit_factor_1,weight_good_loans_repayment_ratio,weight_consistency,weight_recency,risk_rules_factor,ultimate_factor_21,ultimate_factor_7,ultimate_factor_1,limit_21_day,limit_7_day,limit_1_day,adjusted_21_limit,adjusted_7_limit,adjusted_1_limit,final_21_limit,final_7_limit,final_1_limit,blacklist_flag,total_final_21_limit,previous_21_limit,previous_7_limit,previous_1_limit


In [ ]:
def label_model(df):
    """
    function to label model version and track model changes i.e.\
    model index/rank e.g 2022_001,
    model_start_date e.g 2022,2,24 reported as year-month-day,
    model_latest_date e.g today() reported as year-month-day,
    this is full is combined to i,e 2022_001[2022-2-24, 2022-3-24]
    
    Inputs:
    Model start date
    Current latest refresh date for the model
    Model index/rank
    
    Outputs:
    model version that dynamically tracks the dates of refresh for a particulay model 
    
    """
    model_start_date = dt.datetime(2022,5,14)
    model_latest_date = (pd.Timestamp.today()).strftime("%Y-%m-%d")
    model_index = "2022-004"
    
    model_version  = model_index + "[" + model_start_date.strftime("%Y-%m-%d") + "," + " "+ model_latest_date +"]"
    
    return model_version


df["model_version"] = label_model(df)

In [ ]:
#add created at column

model_latest_date = (pd.Timestamp.today()).strftime("%Y-%m-%d %H:%M:%S")

df["created_at"] = model_latest_date

#convert column to timestamp

df["created_at"] = df["created_at"].apply(pd.to_datetime, errors="coerce")


In [ ]:
#add record_added_to_warehouse_on_timestamp column

model_latest_date = (pd.Timestamp.today()).strftime("%Y-%m-%d %H:%M:%S:%f")

df["record_added_to_warehouse_on_timestamp"] = model_latest_date

#convert column to timestamp

#f["created_at"] = df["created_at"].apply(pd.to_datetime, errors="coerce")
#record_added_to_warehouse_on_timestamp

In [ ]:
df.shape

(56816, 72)

In [ ]:
df.head(3)

,mobile_number,store_number,approx_30_days_trx_val,most_recent_trx_date_past_30_days,last_trx_date,expected_trx_days,actual_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,idm_recommendation,idm_limit,is_iprs_validated,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,idm_factor_21,idm_factor_7,idm_factor_1,trading_consistency_bands,loan_count_bands,new_limit_factor_21,new_limit_factor_7,new_limit_factor_1,weight_good_loans_repayment_ratio,weight_consistency,weight_recency,risk_rules_factor,ultimate_factor_21,ultimate_factor_7,ultimate_factor_1,limit_21_day,limit_7_day,limit_1_day,adjusted_21_limit,adjusted_7_limit,adjusted_1_limit,final_21_limit,final_7_limit,final_1_limit,blacklist_flag,total_final_21_limit,previous_21_limit,previous_7_limit,previous_1_limit,model_version,created_at,record_added_to_warehouse_on_timestamp
0,254722978849,822369,156.67,2022-06-18,2022-06-18,1,1,1.00,relax_rules,88,No,0.00,22128407,Approve,0.00,True,NaN,4.00,600.00,7.00,5000.00,5000.00,2017-12-10,2017-12-17,2017-12-10,2017-12-17,-7.00,1.00,closed_early_repayment,NaN,5000.00,2017-12-10,4.00,1.00,1738.00,1,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 7,Band 3,0.30,0.17,0.17,1.00,1.00,0,0.75,0.22,0.13,0.13,35.25,20.56,20.56,0.00,20.56,20.56,0,0,0,0,0.00,0.00,0.00,0.00,"2022-004[2022-05-14, 2022-09-14]",2022-09-14 12:36:14,2022-09-14 12:36:19:241406
1,254723173634,786152,2050.00,2022-06-14,2022-09-08,87,12,0.14,No_rules_relaxed,6,No,0.00,494024,Approve,0.00,True,NaN,1.00,601.00,30.00,12698.00,9206.00,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.00,1.00,written-off_default,NaN,12698.00,2017-12-18,0.00,0.00,1730.00,0,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 1,Band 2,0.00,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,1,0.00,0.00,0.00,0.00,"2022-004[2022-05-14, 2022-09-14]",2022-09-14 12:36:14,2022-09-14 12:36:19:241406
2,254723916436,165978,810233.88,2022-06-13,2022-09-12,92,91,0.99,No_rules_relaxed,2,Yes,1.00,20417564,Approve,0.00,True,NaN,1.00,601.00,30.00,20000.00,12800.00,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.00,1.00,written-off_default,NaN,20000.00,2017-12-18,0.00,0.00,1730.00,0,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 7,Band 2,0.25,0.15,0.15,0.00,1.00,2,0.75,0.19,0.11,0.11,151918.85,91151.31,91151.31,0.00,91151.31,91151.31,0,0,0,1,304500.00,0.00,0.00,0.00,"2022-004[2022-05-14, 2022-09-14]",2022-09-14 12:36:14,2022-09-14 12:36:19:241406


In [ ]:
df[["final_21_limit","final_7_limit","final_1_limit"]].sum()

final_21_limit    147157600
final_7_limit     371855300
final_1_limit     373136400
dtype: int64

In [ ]:
df['idm_recommendation'].value_counts()

Reject     40768
Approve    16048
Name: idm_recommendation, dtype: int64

In [ ]:
print("num of customers who've been scored: {}".format(df["store_number"].nunique()))
print("\n")
print("gross limit allocation for 21 day: {}".format(df["final_21_limit"].sum()))
print("gross num of store nums allocated 21 day limit: {}".format(df[df["final_21_limit"]>0]["store_number"].nunique()))
print("effective limit allocation for 21 day  with blacklist check: {}".format(df.loc[df["blacklist_flag"]==0]["final_21_limit"].sum()))
print("effective num of store nums allocated 21 day limit: {}".format(df.loc[(df["final_21_limit"]>0)&(df["blacklist_flag"]==0)]["store_number"].nunique()))
print("\n")
print("gross limit allocation for 7 day: {}".format(df["final_7_limit"].sum()))
print("gross num of store nums allocated 7 day limit: {}".format(df[df["final_7_limit"]>0]["store_number"].nunique()))
print("effective limit allocation for 7 day  with blacklist check: {}".format(df.loc[df["blacklist_flag"]==0]["final_7_limit"].sum()))
print("effective num of store nums allocated 7 day limit: {}".format(df.loc[(df["final_7_limit"]>0)&(df["blacklist_flag"]==0)]["store_number"].nunique()))
print("\n")
print("gross limit allocation for 1 day: {}".format(df["final_1_limit"].sum()))
print("gross num of store nums allocated 1 day limit: {}".format(df[df["final_1_limit"]>0]["store_number"].nunique()))
print("effective limit allocation for 1 day with blacklist check: {}".format(df.loc[df["blacklist_flag"]==0]["final_1_limit"].sum()))
print("effective num of store nums allocated 1 day limit: {}".format(df.loc[(df["final_1_limit"]>0)&(df["blacklist_flag"]==0)]["store_number"].nunique()))

num of customers who've been scored: 56816


gross limit allocation for 21 day: 147157600
gross num of store nums allocated 21 day limit: 1651
effective limit allocation for 21 day  with blacklist check: 147157600
effective num of store nums allocated 21 day limit: 1651


gross limit allocation for 7 day: 371855300
gross num of store nums allocated 7 day limit: 13589
effective limit allocation for 7 day  with blacklist check: 371716600
effective num of store nums allocated 7 day limit: 13568


gross limit allocation for 1 day: 373136400
gross num of store nums allocated 1 day limit: 15183
effective limit allocation for 1 day with blacklist check: 372996300
effective num of store nums allocated 1 day limit: 15159


In [ ]:
df.head()

,mobile_number,store_number,approx_30_days_trx_val,most_recent_trx_date_past_30_days,last_trx_date,expected_trx_days,actual_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,idm_recommendation,idm_limit,is_iprs_validated,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,idm_factor_21,idm_factor_7,idm_factor_1,trading_consistency_bands,loan_count_bands,new_limit_factor_21,new_limit_factor_7,new_limit_factor_1,weight_good_loans_repayment_ratio,weight_consistency,weight_recency,risk_rules_factor,ultimate_factor_21,ultimate_factor_7,ultimate_factor_1,limit_21_day,limit_7_day,limit_1_day,adjusted_21_limit,adjusted_7_limit,adjusted_1_limit,final_21_limit,final_7_limit,final_1_limit,blacklist_flag,total_final_21_limit,previous_21_limit,previous_7_limit,previous_1_limit,model_version,created_at,record_added_to_warehouse_on_timestamp
0,254722978849,822369,156.67,2022-06-18,2022-06-18,1,1,1.00,relax_rules,88,No,0.00,22128407,Approve,0.00,True,NaN,4.00,600.00,7.00,5000.00,5000.00,2017-12-10,2017-12-17,2017-12-10,2017-12-17,-7.00,1.00,closed_early_repayment,NaN,5000.00,2017-12-10,4.00,1.00,1738.00,1,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 7,Band 3,0.30,0.17,0.17,1.00,1.00,0,0.75,0.22,0.13,0.13,35.25,20.56,20.56,0.00,20.56,20.56,0,0,0,0,0.00,0.00,0.00,0.00,"2022-004[2022-05-14, 2022-09-14]",2022-09-14 12:36:14,2022-09-14 12:36:19:241406
1,254723173634,786152,2050.00,2022-06-14,2022-09-08,87,12,0.14,No_rules_relaxed,6,No,0.00,494024,Approve,0.00,True,NaN,1.00,601.00,30.00,12698.00,9206.00,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.00,1.00,written-off_default,NaN,12698.00,2017-12-18,0.00,0.00,1730.00,0,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 1,Band 2,0.00,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,1,0.00,0.00,0.00,0.00,"2022-004[2022-05-14, 2022-09-14]",2022-09-14 12:36:14,2022-09-14 12:36:19:241406
2,254723916436,165978,810233.88,2022-06-13,2022-09-12,92,91,0.99,No_rules_relaxed,2,Yes,1.00,20417564,Approve,0.00,True,NaN,1.00,601.00,30.00,20000.00,12800.00,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.00,1.00,written-off_default,NaN,20000.00,2017-12-18,0.00,0.00,1730.00,0,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 7,Band 2,0.25,0.15,0.15,0.00,1.00,2,0.75,0.19,0.11,0.11,151918.85,91151.31,91151.31,0.00,91151.31,91151.31,0,0,0,1,304500.00,0.00,0.00,0.00,"2022-004[2022-05-14, 2022-09-14]",2022-09-14 12:36:14,2022-09-14 12:36:19:241406
3,254720475133,745142,160453.67,2022-06-13,2022-09-12,92,89,0.97,No_rules_relaxed,2,Yes,1.00,22893884,Reject,0.00,True,NaN,1.00,601.00,30.00,28684.00,0.00,2017-12-21,2018-01-20,2019-12-31,2018-01-20,710.00,1.00,written-off_default,NaN,28684.00,2017-12-21,0.00,0.00,1727.00,0,0.00,0.35,0.12,0.12,0.70,0.71,0.71,Band 7,Band 2,0.25,0.15,0.15,0.00,1.00,2,0.75,0.13,0.08,0.08,21059.54,12741.91,12741.91,0.00,12741.91,12741.91,0,0,0,1,27200.00,0.00,0.00,0.00,"2022-004[2022-05-14, 2022-09-14]",2022-09-14 12:36:14,2022-09-14 12:36:19:241406
4,254724756423,978161,366.67,2022-07-12,2022-07-15,4,2,0.50,No_rules_relaxed,61,No,0.00,13261352,Approve,0.00,True,NaN,2.00,601.00,30.00,28600.00,0.00,2018-01-07,2018-02-06,2018-12-31,2018-02-06,328.00,1.00,written-off_default,NaN,28680.00,2017-12-19,1.00,0.50,1710.00,0,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 3,Band 2,0.05,0.07,0.07,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,1,0.00,0.00,0.00,0.00,"2022-004[2022-05-14, 2022-09-14]",2022-09-14 12:36:14,2022-09-14 12:36:19:241406


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56816 entries, 0 to 56819
Data columns (total 72 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   mobile_number                           44564 non-null  object        
 1   store_number                            56816 non-null  object        
 2   approx_30_days_trx_val                  56816 non-null  float64       
 3   most_recent_trx_date_past_30_days       56816 non-null  datetime64[ns]
 4   last_trx_date                           56816 non-null  datetime64[ns]
 5   expected_trx_days                       56816 non-null  int64         
 6   actual_trx_days                         56816 non-null  int64         
 7   page_active_days                        56816 non-null  float64       
 8   inference_col                           56816 non-null  object        
 9   days_since_last_trx                     56816 non-

In [ ]:
cols_fillna = ["is_iprs_validated"]
# replace 'NaN' with zero in these columns
for col in cols_fillna:
    df[col].fillna('False',inplace=True)

In [ ]:
df['is_iprs_validated'].value_counts()

True     44559
False    12257
Name: is_iprs_validated, dtype: int64

In [ ]:
# Limit adjustment rule 1: Merchants that have DPD 30 plus on loans clients get a 50% limit reduction across all limits (1-day, 7-day and 21-day, subject to product minimum rules) 
def calc_final_21_limit(df):
    
    is_iprs_validated = df["is_iprs_validated"]
    final_21_limit = df["final_21_limit"]
    
    if is_iprs_validated == 'False':
        return 0
    else:
        return final_21_limit
    
    
def calc_final_7_limit(df):
    
    is_iprs_validated = df["is_iprs_validated"]
    final_7_limit = df["final_7_limit"]
    
    if is_iprs_validated == 'False':
        return 0
    else:
        return final_7_limit    
    
    
def calc_final_1_limit(df):
    
    is_iprs_validated = df["is_iprs_validated"]
    final_1_limit = df["final_1_limit"]
    
    if is_iprs_validated == 'False':
        return 0
    else:
        return final_1_limit  

In [ ]:
df['final_21_limit'] = df.apply(lambda x: calc_final_21_limit(x), axis = 1)
df['final_7_limit'] = df.apply(lambda x: calc_final_7_limit(x), axis = 1)
df['final_1_limit'] = df.apply(lambda x: calc_final_1_limit(x), axis = 1)

In [ ]:
df[["final_21_limit","final_7_limit","final_1_limit"]].sum()

final_21_limit    147157600
final_7_limit     371855300
final_1_limit     373136400
dtype: int64

In [ ]:
print("num of customers who've been scored: {}".format(df["store_number"].nunique()))
print("\n")
print("gross limit allocation for 21 day: {}".format(df["final_21_limit"].sum()))
print("gross num of store nums allocated 21 day limit: {}".format(df[df["final_21_limit"]>0]["store_number"].nunique()))
print("effective limit allocation for 21 day  with blacklist check: {}".format(df.loc[df["blacklist_flag"]==0]["final_21_limit"].sum()))
print("effective num of store nums allocated 21 day limit: {}".format(df.loc[(df["final_21_limit"]>0)&(df["blacklist_flag"]==0)]["store_number"].nunique()))
print("\n")
print("gross limit allocation for 7 day: {}".format(df["final_7_limit"].sum()))
print("gross num of store nums allocated 7 day limit: {}".format(df[df["final_7_limit"]>0]["store_number"].nunique()))
print("effective limit allocation for 7 day  with blacklist check: {}".format(df.loc[df["blacklist_flag"]==0]["final_7_limit"].sum()))
print("effective num of store nums allocated 7 day limit: {}".format(df.loc[(df["final_7_limit"]>0)&(df["blacklist_flag"]==0)]["store_number"].nunique()))
print("\n")
print("gross limit allocation for 1 day: {}".format(df["final_1_limit"].sum()))
print("gross num of store nums allocated 1 day limit: {}".format(df[df["final_1_limit"]>0]["store_number"].nunique()))
print("effective limit allocation for 1 day with blacklist check: {}".format(df.loc[df["blacklist_flag"]==0]["final_1_limit"].sum()))
print("effective num of store nums allocated 1 day limit: {}".format(df.loc[(df["final_1_limit"]>0)&(df["blacklist_flag"]==0)]["store_number"].nunique()))

num of customers who've been scored: 56816


gross limit allocation for 21 day: 147157600
gross num of store nums allocated 21 day limit: 1651
effective limit allocation for 21 day  with blacklist check: 147157600
effective num of store nums allocated 21 day limit: 1651


gross limit allocation for 7 day: 371855300
gross num of store nums allocated 7 day limit: 13589
effective limit allocation for 7 day  with blacklist check: 371716600
effective num of store nums allocated 7 day limit: 13568


gross limit allocation for 1 day: 373136400
gross num of store nums allocated 1 day limit: 15183
effective limit allocation for 1 day with blacklist check: 372996300
effective num of store nums allocated 1 day limit: 15159


In [ ]:
# defining a function for the limit caps
def calc_final_21_limit(df):

    final_21_limit = df['final_21_limit']
    days_past_due = df['days_past_due']
  
    if days_past_due > 15:
        return 0
    else:
        return final_21_limit

    

def calc_final_7_limit(df):

    final_7_limit = df['final_7_limit']
    days_past_due = df['days_past_due']
  
    if days_past_due > 15:
        return 0
    else:
        return final_7_limit



def calc_final_1_limit(df):

    final_1_limit = df['final_1_limit']
    days_past_due = df['days_past_due']
  
    if days_past_due > 15:
        return 0
    else:
        return final_1_limit

In [ ]:
df['final_21_limit'] = df.apply(lambda x: calc_final_21_limit(x), axis = 1)
df['final_7_limit'] = df.apply(lambda x: calc_final_7_limit(x), axis = 1)
df['final_1_limit'] = df.apply(lambda x: calc_final_1_limit(x), axis = 1)

In [ ]:
df[["final_21_limit","final_7_limit","final_1_limit"]].sum()

final_21_limit    145646300
final_7_limit     365911000
final_1_limit     367177600
dtype: int64

In [ ]:
print("num of customers who've been scored: {}".format(df["store_number"].nunique()))
print("\n")
print("gross limit allocation for 21 day: {}".format(df["final_21_limit"].sum()))
print("gross num of store nums allocated 21 day limit: {}".format(df[df["final_21_limit"]>0]["store_number"].nunique()))
print("effective limit allocation for 21 day  with blacklist check: {}".format(df.loc[df["blacklist_flag"]==0]["final_21_limit"].sum()))
print("effective num of store nums allocated 21 day limit: {}".format(df.loc[(df["final_21_limit"]>0)&(df["blacklist_flag"]==0)]["store_number"].nunique()))
print("\n")
print("gross limit allocation for 7 day: {}".format(df["final_7_limit"].sum()))
print("gross num of store nums allocated 7 day limit: {}".format(df[df["final_7_limit"]>0]["store_number"].nunique()))
print("effective limit allocation for 7 day  with blacklist check: {}".format(df.loc[df["blacklist_flag"]==0]["final_7_limit"].sum()))
print("effective num of store nums allocated 7 day limit: {}".format(df.loc[(df["final_7_limit"]>0)&(df["blacklist_flag"]==0)]["store_number"].nunique()))
print("\n")
print("gross limit allocation for 1 day: {}".format(df["final_1_limit"].sum()))
print("gross num of store nums allocated 1 day limit: {}".format(df[df["final_1_limit"]>0]["store_number"].nunique()))
print("effective limit allocation for 1 day with blacklist check: {}".format(df.loc[df["blacklist_flag"]==0]["final_1_limit"].sum()))
print("effective num of store nums allocated 1 day limit: {}".format(df.loc[(df["final_1_limit"]>0)&(df["blacklist_flag"]==0)]["store_number"].nunique()))

num of customers who've been scored: 56816


gross limit allocation for 21 day: 145646300
gross num of store nums allocated 21 day limit: 1611
effective limit allocation for 21 day  with blacklist check: 145646300
effective num of store nums allocated 21 day limit: 1611


gross limit allocation for 7 day: 365911000
gross num of store nums allocated 7 day limit: 13345
effective limit allocation for 7 day  with blacklist check: 365911000
effective num of store nums allocated 7 day limit: 13345


gross limit allocation for 1 day: 367177600
gross num of store nums allocated 1 day limit: 14921
effective limit allocation for 1 day with blacklist check: 367177600
effective num of store nums allocated 1 day limit: 14921


In [ ]:
df.to_excel("C:\\Project_summaries\\Bloom\\Bloom all_loans\\20220914\\Analysis_summaries\\Limits_refresh_summary_20220914_multiple_products.xlsx")

In [ ]:
pd.DataFrame(bloom_customers_no_till, columns = ["national_id"]).to_excel("C:\\Project_summaries\\Bloom\\Bloom all_loans\\20220908\\Analysis_summaries\\"+"bloom_customers_no_till_9.xlsx")

In [ ]:
excluded_first_time_df.to_excel("C:\\Project_summaries\\Bloom\\Bloom all_loans\\20220908\\Analysis_summaries\\excluded_new_customers_20220908.xlsx")

---
#### Add reasons for rejection

In [ ]:
df['store_number'] = df['store_number'].astype(str)

In [ ]:
host = '157.245.248.249'
port = int(5432)
dbname = 'ubuntu'
user = 'jacklinengenia'
password = 'x3MX&8#!'


def get_query_results_postgres(df):
    with psycopg2.connect(host = host,
                          port = port,
                          database = dbname,
                          user = user,
                          password = password) as conn:
        sql = f"select store_number from bloomlive.client_summary_view csv2 where store_number not in {tuple(df['store_number'].tolist())}"
        df = pd.read_sql(sql, conn)
        
    return df

    conn.close()

In [ ]:
# loading data from scoring results table
customer_details = get_query_results_postgres(df)

customer_details.head(3)

In [ ]:
customer_details = customer_details.drop_duplicates(subset=['store_number'], keep='last')

customer_details.shape

In [ ]:
# removing the test accounts from dataframe
removed_lst = ['573691', '219091', '884766', '735346', '5009587']
customer_details = customer_details[~customer_details['store_number'].isin(removed_lst)]

In [ ]:
customer_details['final_21_limit'] = 0
customer_details['final_7_limit'] = 0
customer_details['final_1_limit'] = 0
customer_details['blacklist_flag'] = 0

customer_details.head()

In [ ]:
def label_model(df):
    """
    function to label model version and track model changes i.e.\
    model index/rank e.g 2022_001,
    model_start_date e.g 2022,2,24 reported as year-month-day,
    model_latest_date e.g today() reported as year-month-day,
    this is full is combined to i,e 2022_001[2022-2-24, 2022-3-24]
    
    Inputs:
    Model start date
    Current latest refresh date for the model
    Model index/rank
    
    Outputs:
    model version that dynamically tracks the dates of refresh for a particulay model 
    
    """
    model_start_date = dt.datetime(2022,5,14)
    model_latest_date = (pd.Timestamp.today()).strftime("%Y-%m-%d")
    model_index = "2022-004"
    
    model_version  = model_index + "[" + model_start_date.strftime("%Y-%m-%d") + "," + " "+ model_latest_date +"]"
    
    return model_version


customer_details["model_version"] = label_model(df)

In [ ]:
#add created at column

model_latest_date = (pd.Timestamp.today()).strftime("%Y-%m-%d %H:%M:%S")

customer_details["created_at"] = model_latest_date

#convert column to timestamp

customer_details["created_at"] = customer_details["created_at"].apply(pd.to_datetime, errors="coerce")

In [ ]:
#add record_added_to_warehouse_on_timestamp column

model_latest_date = (pd.Timestamp.today()).strftime("%Y-%m-%d %H:%M:%S:%f")

customer_details["record_added_to_warehouse_on_timestamp"] = model_latest_date

#convert column to timestamp

In [ ]:
df['is_iprs_validated'] = df['is_iprs_validated'].astype(str)

In [ ]:
def rejection_reasons(df):
    """
    """
    good_loans_repayment_ratio = df["good_loans_repayment_ratio"]
    days_past_due = df["days_past_due"]
    till_recency = df["transacted_last_5_days"]
    till_consistency = df["page_active_days"]
    inference_col = df["inference_col"]
    idm_recommendation = df["idm_recommendation"]
    loan_count = df["loan_count"]
    num_days_since_last_disbursement = df["num_days_since_last_disbursement"]
    initial_21_day_limit = df["limit_21_day"]
    initial_7_day_limit = df["limit_7_day"]
    initial_1_day_limit = df["limit_1_day"]
    is_iprs_validated = df["is_iprs_validated"]
    
    
    repayment_ratio_response = "bad repayment history:A1"
    dpd_response = "bad repayment history:A2"
    till_recency_response = "lower than expected trading activity:B1"
    till_consistency_response = "lower than expected trading activity:B2"
    idm_recommendation_response = "inadequate CRB risk profile:C1"
    num_days_since_last_disbursement_response = "insufficient recent credit activity:D1"
    trivial_limits_cut_off = "limit assigned less than product thresholds:E1"
    kyc_check = "limits zeroized due to incomplete or invalid KYC:H1"
    
    
    conditions = [
        loan_count.gt(0) & good_loans_repayment_ratio.lt(0.6),
        days_past_due.gt(30) & inference_col.str.match("No_rules_relaxed"),
        days_past_due.gt(41) & inference_col.str.match("relax_rules"),
        till_recency.str.match("No"),
        till_consistency.lt(0.7) & inference_col.str.match("No_rules_relaxed"),
        till_consistency.lt(0.49) & inference_col.str.match("relax_rules"),
        loan_count.lt(6) & idm_recommendation.str.match("Reject"),
        num_days_since_last_disbursement.gt(180),
        initial_21_day_limit.lt(1000) | initial_7_day_limit.lt(1000) | initial_7_day_limit.lt(200),
        is_iprs_validated.str.match("False")
    ]
    
    choices = [
        repayment_ratio_response,
        dpd_response,
        dpd_response,
        till_recency_response,
        till_consistency_response,
        till_consistency_response,
        idm_recommendation_response,
        num_days_since_last_disbursement_response,
        trivial_limits_cut_off,
        kyc_check
    ]
    
    reasons_col = np.select(conditions, choices)
    
    return reasons_col

#apply function
df["rules_summary_narration"] = rejection_reasons(df)

#fix unassigned rows
df.loc[(df["rules_summary_narration"]=="0")&(df["blacklist_flag"]==1), "rules_summary_narration"] = "part of Mifos recon list:F1"
df.loc[(df["rules_summary_narration"]=="0")&(df["blacklist_flag"]==0), "rules_summary_narration"] = "all rules passed:G1"

In [ ]:
df["rules_summary_narration"].unique()

array(['insufficient recent credit activity:D1',
       'bad repayment history:A1',
       'lower than expected trading activity:B2',
       'inadequate CRB risk profile:C1',
       'lower than expected trading activity:B1',
       'bad repayment history:A2', 'all rules passed:G1',
       'limit assigned less than product thresholds:E1',
       'limits zeroized due to incomplete or invalid KYC:H1'],
      dtype=object)

In [ ]:
#split rules summary narration column to get split of narration and code

df[["rules_summary_narration","limit_reason"]] = df["rules_summary_narration"].astype("str").str.split(":", expand=True)

In [ ]:
df.head(3)

,mobile_number,store_number,approx_30_days_trx_val,most_recent_trx_date_past_30_days,last_trx_date,expected_trx_days,actual_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,idm_recommendation,idm_limit,is_iprs_validated,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,days_past_due,bloom_version,loan_repayment_status,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,idm_factor_21,idm_factor_7,idm_factor_1,trading_consistency_bands,loan_count_bands,new_limit_factor_21,new_limit_factor_7,new_limit_factor_1,weight_consistency,weight_good_loans_repayment_ratio,risk_rules_factor,ultimate_factor_21,ultimate_factor_7,ultimate_factor_1,limit_21_day,limit_7_day,limit_1_day,adjusted_21_limit,adjusted_7_limit,adjusted_1_limit,final_21_limit,final_7_limit,final_1_limit,blacklist_flag,total_final_21_limit,previous_21_limit,previous_7_limit,previous_1_limit,model_version,created_at,record_added_to_warehouse_on_timestamp,rules_summary_narration,limit_reason
0,254722978849,822369,2020.00,2022-07-21,2022-07-21,1,1,1.00,relax_rules,5,Yes,1.00,22128407,Approve,0.00,True,254722978849.00,4.00,600.00,7.00,5000.00,5000.00,2017-12-10,2017-12-17,2017-12-10,2017-12-17,-7.00,1.00,closed_early_repayment,5000.00,2017-12-10,4.00,1.00,1688.00,1.00,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 7,Band 3,0.30,0.17,0.17,1.00,1.00,1.00,0.30,0.17,0.17,606.00,353.50,353.50,0.00,353.50,353.50,0,0,0,0,0.00,0,0,0,"2022-004[2022-05-14, 2022-07-27]",2022-07-27 11:58:00,2022-07-27 11:58:05:663659,insufficient recent credit activity,D1
1,254723916436,165978,905554.07,2022-06-27,2022-07-25,29,28,0.97,No_rules_relaxed,1,Yes,1.00,20417564,Approve,0.00,True,254723916436.00,1.00,601.00,30.00,20000.00,12800.00,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.00,1.00,written-off_default,20000.00,2017-12-18,0.00,0.00,1680.00,0.00,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 7,Band 2,0.25,0.15,0.15,1.00,0.00,0.50,0.12,0.07,0.07,113194.26,67916.56,67916.56,0.00,67916.56,67916.56,0,23000,23000,1,304500.00,0,23000,23000,"2022-004[2022-05-14, 2022-07-27]",2022-07-27 11:58:00,2022-07-27 11:58:05:663659,bad repayment history,A1
2,254723173634,786152,1370.00,2022-07-15,2022-07-25,11,2,0.18,No_rules_relaxed,1,Yes,1.00,494024,Approve,0.00,True,254723173634.00,1.00,601.00,30.00,12698.00,9206.00,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.00,1.00,written-off_default,12698.00,2017-12-18,0.00,0.00,1680.00,0.00,0.00,0.50,0.17,0.17,1.00,1.00,1.00,Band 1,Band 2,0.00,0.00,0.00,0.00,0.00,0.25,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,1,0.00,0,0,0,"2022-004[2022-05-14, 2022-07-27]",2022-07-27 11:58:00,2022-07-27 11:58:05:663659,bad repayment history,A1


In [ ]:
# including the test accounts with new limits assigned
df2 = pd.DataFrame({'store_number': ['573691', '219091', '884766', '735346', '5009587'],
                   'blacklist_flag': [0, 0, 0, 0, 0],
                   'final_21_limit': [1000, 1000, 1000, 1000, 1000],
                   'final_7_limit': [500, 500, 500, 500, 1000],
                   'final_1_limit': [300, 300, 300, 300, 1000]})

# adding the test accounts into the dataframe
#df = df.append(df2,ignore_index=True)

In [ ]:
df2.head(3)

,store_number,blacklist_flag,final_21_limit,final_7_limit,final_1_limit
0,573691,0,1000,500,300
1,219091,0,1000,500,300
2,884766,0,1000,500,300


In [ ]:
def label_model(df):
    """
    function to label model version and track model changes i.e.\
    model index/rank e.g 2022_001,
    model_start_date e.g 2022,2,24 reported as year-month-day,
    model_latest_date e.g today() reported as year-month-day,
    this is full is combined to i,e 2022_001[2022-2-24, 2022-3-24]
    
    Inputs:
    Model start date
    Current latest refresh date for the model
    Model index/rank
    
    Outputs:
    model version that dynamically tracks the dates of refresh for a particulay model 
    
    """
    model_start_date = dt.datetime(2022,5,14)
    model_latest_date = (pd.Timestamp.today()).strftime("%Y-%m-%d")
    model_index = "2022-004"
    
    model_version  = model_index + "[" + model_start_date.strftime("%Y-%m-%d") + "," + " "+ model_latest_date +"]"
    
    return model_version


df2["model_version"] = label_model(df)

In [ ]:
#add created at column

model_latest_date = (pd.Timestamp.today()).strftime("%Y-%m-%d %H:%M:%S")

df2["created_at"] = model_latest_date

#convert column to timestamp

df2["created_at"] = df2["created_at"].apply(pd.to_datetime, errors="coerce")

In [ ]:
#add record_added_to_warehouse_on_timestamp column

model_latest_date = (pd.Timestamp.today()).strftime("%Y-%m-%d %H:%M:%S:%f")

df2["record_added_to_warehouse_on_timestamp"] = model_latest_date

#convert column to timestamp

#f["created_at"] = df["created_at"].apply(pd.to_datetime, errors="coerce")
#record_added_to_warehouse_on_timestamp

In [ ]:
df2.head(3)

,store_number,blacklist_flag,final_21_limit,final_7_limit,final_1_limit,model_version,created_at,record_added_to_warehouse_on_timestamp
0,573691,0,1000,500,300,"2022-004[2022-05-14, 2022-07-27]",2022-07-27 12:00:47,2022-07-27 12:00:47:196055
1,219091,0,1000,500,300,"2022-004[2022-05-14, 2022-07-27]",2022-07-27 12:00:47,2022-07-27 12:00:47:196055
2,884766,0,1000,500,300,"2022-004[2022-05-14, 2022-07-27]",2022-07-27 12:00:47,2022-07-27 12:00:47:196055


In [ ]:
# adding the test accounts into the dataframe
df = df.append(df2,ignore_index=True)

In [ ]:
# adding the test accounts into the dataframe
df = df.append(customer_details,ignore_index=True)

---
#### Push limits to DB ==> bloomlive.scoring_results

In [ ]:
#get copy of df
scoring_results = df.copy()


#trim df to only match DB column names
db_cols = [
    "store_number", "national_id","final_21_limit",
    "final_7_limit","final_1_limit", "idm_recommendation",
    "limit_factor_21", "limit_factor_7", "limit_factor_1",
    "model_version", "blacklist_flag", "created_at", "record_added_to_warehouse_on_timestamp","limit_reason"
]

scoring_results = scoring_results[db_cols]

In [ ]:
#convert store_number col to string type

scoring_results["store_number"] = scoring_results["store_number"].astype(str)

In [ ]:
scoring_results.head(3)

,store_number,national_id,final_21_limit,final_7_limit,final_1_limit,idm_recommendation,limit_factor_21,limit_factor_7,limit_factor_1,model_version,blacklist_flag,created_at,record_added_to_warehouse_on_timestamp,limit_reason
0,822369,822369,0,0,0,Reject,0.35,0.12,0.12,"2022-004[2022-05-14, 2022-06-29]",0,2022-06-29 18:41:13,2022-06-29 18:41:17:704455,B1
1,786152,786152,0,0,0,Approve,0.50,0.17,0.17,"2022-004[2022-05-14, 2022-06-29]",1,2022-06-29 18:41:13,2022-06-29 18:41:17:704455,A1
2,165978,165978,0,23000,23000,Reject,0.35,0.12,0.12,"2022-004[2022-05-14, 2022-06-29]",1,2022-06-29 18:41:13,2022-06-29 18:41:17:704455,A1


In [ ]:
scoring_results.shape

(51055, 14)

In [ ]:
#function to write pandas scoring results df to db table 
def write_results_to_db(conn, df_final, table):
  
    tuples = [tuple(x) for x in df_final.to_numpy()]
  
    cols = ','.join(list(df_final.columns))
    # SQL query to execute
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("the dataframe is inserted")
    cursor.close()
  
  
conn = psycopg2.connect(
    database="ubuntu", user="jacklinengenia", password="x3MX&8#!", host="157.245.248.249", port='5432'
)
  
df_final = scoring_results.copy()
  
write_results_to_db(conn, df_final, "bloomlive.scoring_results")

the dataframe is inserted


In [ ]:
import requests
import datetime as dt
from dateutil import tz
from tzlocal import get_localzone
local_tz = get_localzone()
import datetime

In [ ]:
model_version = '2022-004[2022-05-14, 2022-06-30]'

def trigger_bloom_limit_refresh_push(model_version, is_initial_run):
    response = requests.post(
        url='https://192.241.150.164/airflow/api/v1/dags/Bloom_limit_refresh_push/dagRuns',
        headers={
            'Content-type': 'application/json',
            'Accept': 'application/json'
        },
        json={
            "execution_date": str(datetime.datetime.now().replace(tzinfo=local_tz)),
            "conf": {
                'model_version': model_version,
                'is_initial_run': is_initial_run,
            }
        },
        auth=requests.auth.HTTPBasicAuth("jackline.ngenia@asantefinancegroup.com", '=tR"5TK(9)'),
        verify=False
    )
    
    print(response.status_code)
    print(response.text)

trigger_bloom_limit_refresh_push(model_version=model_version, is_initial_run='true')

C:\Users\USER\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.241.150.164'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


200
{
  "conf": {
    "is_initial_run": "true",
    "model_version": "2022-004[2022-05-14, 2022-06-30]"
  },
  "dag_id": "Bloom_limit_refresh_push",
  "dag_run_id": "manual__2022-07-20T09:22:13.709387+03:00",
  "end_date": null,
  "execution_date": "2022-07-20T09:22:13.709387+03:00",
  "external_trigger": true,
  "logical_date": "2022-07-20T09:22:13.709387+03:00",
  "start_date": null,
  "state": "queued"
}



In [ ]:
#convert store_number col to string type
df['final_21_limit'] = df['final_21_limit'].fillna(0)
df['final_7_limit'] = df['final_7_limit'].fillna(0)
df['final_1_limit'] = df['final_1_limit'].fillna(0)

df["most_recent_trx_date_past_30_days"] = df["most_recent_trx_date_past_30_days"].astype(str)
df["last_trx_date"] = df["last_trx_date"].astype(str)
df["national_id"] = df["national_id"].astype(str)
df["disbursed_on_date"] = df["disbursed_on_date"].astype(str)
df["expected_matured_on_date"] = df["expected_matured_on_date"].astype(str)
df["closed_on_date"] = df["closed_on_date"].astype(str)
df["due_date_fixed"] = df["due_date_fixed"].astype(str)
df["max_loan_disbursement_date"] = df["max_loan_disbursement_date"].astype(str)
df["due_date_fixed"] = df["due_date_fixed"].astype(str)
df["due_date_fixed"] = df["due_date_fixed"].astype(str)
df["created_at"] = df["created_at"].astype(str)
df["record_added_to_warehouse_on_timestamp"] = df["record_added_to_warehouse_on_timestamp"].astype(str)

df["adjusted_loan_count"] = df["adjusted_loan_count"].astype(float)
df["blacklist_flag"] = df["blacklist_flag"].astype(float)
df["due_date_fixed"] = df["due_date_fixed"].astype(str)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51055 entries, 0 to 51054
Data columns (total 74 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Unnamed: 0                              51050 non-null  float64
 1   store_number                            51055 non-null  object 
 2   approx_30_days_trx_val                  51050 non-null  float64
 3   most_recent_trx_date_past_30_days       51055 non-null  object 
 4   last_trx_date                           51055 non-null  object 
 5   expected_trx_days                       51050 non-null  float64
 6   actual_trx_days                         51050 non-null  float64
 7   page_active_days                        51050 non-null  float64
 8   inference_col                           51050 non-null  object 
 9   days_since_last_trx                     51050 non-null  float64
 10  transacted_last_5_days                  51050 non-null  ob

In [ ]:
df.shape

(51055, 74)

In [ ]:
#get copy of df
df = df.copy()


#trim df to only match DB column names
db_cols = [
    "store_number", "approx_30_days_trx_val", "most_recent_trx_date_past_30_days", "last_trx_date","expected_trx_days", "actual_trx_days", "page_active_days",
    "days_since_last_trx", "transacted_last_5_days", "weight_till_recency", "national_id", "client_mobile_number", "loan_count", "loan_status", "term_frequency",
    "principal_disbursed",  "principal_repaid", "disbursed_on_date", "expected_matured_on_date", "closed_on_date", "due_date_fixed", "days_past_due", "bloom_version",
    "loan_repayment_status", "max_principal_amount", "max_loan_disbursement_date", "count_good_loans", "good_loans_repayment_ratio", "num_days_since_last_disbursement",
    "weight_dpd", "adjusted_loan_count", "limit_21_day", "limit_7_day", "limit_1_day", "adjusted_21_limit", "adjusted_7_limit", "adjusted_1_limit", "final_21_limit",
    "final_7_limit", "final_1_limit", "model_version", "blacklist_flag", "limit_factor_21", "limit_factor_7", "limit_factor_1", "created_at", 
    "record_added_to_warehouse_on_timestamp"
]

df = df[db_cols]

In [ ]:
df.shape

(51055, 47)

In [ ]:
df.columns

Index(['store_number', 'approx_30_days_trx_val',
       'most_recent_trx_date_past_30_days', 'last_trx_date',
       'expected_trx_days', 'actual_trx_days', 'page_active_days',
       'days_since_last_trx', 'transacted_last_5_days', 'weight_till_recency',
       'national_id', 'client_mobile_number', 'loan_count', 'loan_status',
       'term_frequency', 'principal_disbursed', 'principal_repaid',
       'disbursed_on_date', 'expected_matured_on_date', 'closed_on_date',
       'due_date_fixed', 'days_past_due', 'bloom_version',
       'loan_repayment_status', 'max_principal_amount',
       'max_loan_disbursement_date', 'count_good_loans',
       'good_loans_repayment_ratio', 'num_days_since_last_disbursement',
       'weight_dpd', 'adjusted_loan_count', 'limit_21_day', 'limit_7_day',
       'limit_1_day', 'adjusted_21_limit', 'adjusted_7_limit',
       'adjusted_1_limit', 'final_21_limit', 'final_7_limit', 'final_1_limit',
       'model_version', 'blacklist_flag', 'limit_factor_21', 'lim

In [ ]:
#db connection libraries
import psycopg2
from sqlalchemy import create_engine
import psycopg2.extras as extras

In [ ]:
#function to write pandas scoring results df to db table 
def write_results_to_db(conn, df_final, table):
  
    tuples = [tuple(x) for x in df_final.to_numpy()]
  
    cols = ','.join(list(df_final.columns))
    # SQL query to execute
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("the dataframe is inserted")
    cursor.close()
  
  
conn = psycopg2.connect(
    database="ubuntu", user="jacklinengenia", password="x3MX&8#!", host="157.245.248.249", port='5432'
)
  
df_final = scoring_results.copy()
  
write_results_to_db(conn, df_final, "bloomlive.temp_scoring_summaries")

Error: column "idm_recommendation" of relation "temp_scoring_summaries" does not exist
LINE 1: ...nal_id,final_21_limit,final_7_limit,final_1_limit,idm_recomm...
                                                             ^



1